In [194]:
from tabula import read_pdf
import pandas as pd

# Path to your PDF file
file_path = '/Users/jaredboretsky/Documents/concordia-bootcamps/ds-final_project/CFL_game_reports_2023/2023_CFL_Game_Analysis_Report_Wk_4.pdf'
# Use read_pdf to extract tables into a list of DataFrame objects
dfs = read_pdf(file_path, pages='all', multiple_tables=True, stream=True)

# Each table is a DataFrame, you can process them individually
# for df in dfs:
#     print(df.head())  # Print the first few rows of each table



In [2]:
def split_column(df, column_name):
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in DataFrame")

    # Split the header name to get new column names
    new_column_names = column_name.split()

    # Splitting the column based on spaces
    new_columns = df[column_name].str.split(' ', expand=True)

    # Check if the number of new columns matches the header parts
    if len(new_column_names) != new_columns.shape[1]:
        raise ValueError("The number of words in the header does not match the number of values in the rows")

    # Naming the new columns
    new_columns.columns = new_column_names

    # Dropping the original column and adding the new columns
    df.drop(columns=[column_name], inplace=True)
    df = pd.concat([df, new_columns], axis=1)

    return df

def set_first_row_as_header(df):
    if df.empty:
        raise ValueError("DataFrame is empty")

    # Set the first row as the column names
    new_header = df.iloc[0]  # Grab the first row for the header
    df = df[1:]  # Take the data less the header row
    df.columns = new_header  # Set the header row as the df header

    # Reset the index of the DataFrame
    df.reset_index(drop=True, inplace=True)

    return df

def combine_header_and_first_row_with_space(df, column_name):
    # Check if the column exists in the DataFrame
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in DataFrame")

    # Split the header and the first row on spaces
    header_words = column_name.split()
    first_row_words = df.iloc[0, df.columns.get_loc(column_name)].split()

    # Check if both lists have the same length
    if len(header_words) != len(first_row_words):
        raise ValueError("The header and the first row must have the same number of space-separated words")

    # Combine words with underscore
    combined_words = ['{}_{}'.format(h, f) for h, f in zip(header_words, first_row_words)]

    # Create a new single string for the new column name
    new_column_name = ' '.join(combined_words)

    # Update the DataFrame
    df.rename(columns={column_name: new_column_name}, inplace=True)
    df.iloc[0, df.columns.get_loc(new_column_name)] = new_column_name

    return df
def combine_header_and_first_row(df, column_name):
    
    # Check if the column exists in the DataFrame
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in DataFrame")

    # Get the header name and the first row values
    header_name = column_name
    first_row_values = df.iloc[0, df.columns.get_loc(column_name)].split()

    # Append the header name to each word in the first row
    appended_values = ['{}_{}'.format(header_name, value) for value in first_row_values]

    # Update the first row in the DataFrame
    df.iloc[0, df.columns.get_loc(column_name)] = ' '.join(appended_values)

    return df



In [2319]:
#clean first df

#drop unwanted columns
cols_to_drop = dfs[0].columns[[4, 5, 6]]
dfs[0].drop(cols_to_drop, axis=1, inplace=True)

#set first row as header
dfs[0] = set_first_row_as_header(dfs[0])

#rename column
dfs[0].rename(columns={'- RECORD - HOME AWAY': 'Record Record Home Home Away Away'}, inplace=True)
dfs[0].rename(columns={'SCORING':'Scoring'}, inplace=True)

dfs[0] = combine_header_and_first_row(dfs[0], 'Scoring')

#set first row as header
dfs[0] = set_first_row_as_header(dfs[0])

dfs[0].rename(columns={'W L Pct W L W L': 'Record_W Record_L Pct Home_W Home_L Away_W Away_L'}, inplace=True)

#split columns
dfs[0] = split_column(dfs[0], 'Record_W Record_L Pct Home_W Home_L Away_W Away_L')
dfs[0] = split_column(dfs[0], 'Scoring_PF Scoring_PA Scoring_Diff')

dfs[0]

,TEAM,GP,Record_W,Record_L,Pct,Home_W,Home_L,Away_W,Away_L,Scoring_PF,Scoring_PA,Scoring_Diff
0,BC-x,18,12,6,.667,6,3,6,3,495,439,+56
1,Calgary,18,6,12,.333,3,6,3,6,412,471,-59
2,Edmonton,18,4,14,.222,2,7,2,7,367,517,-150
3,Hamilton-x,18,8,10,.444,3,6,5,4,408,461,-53
4,Montréal-x,18,11,7,.611,5,4,6,3,442,392,+50
5,Ottawa,18,4,14,.222,3,6,1,8,415,507,-92
6,Saskatchewan,18,6,12,.333,4,5,2,7,387,551,-164
7,Toronto-x,18,16,2,.889,9,0,7,2,591,396,+195
8,Winnipeg-x,18,14,4,.778,8,1,6,3,594,377,+217
9,2023 Total,81,81,81,43,38,38,43,None,50.8,None,None


In [2320]:
#clean scoring breakdown df
#only looking at team data not league year by year so skip to dfs[2]






#pop unneeded df from list
item = dfs.pop(2)

#drop last column
column_to_drop = '2023 CFL-WIDE / HISTORY'
dfs[2] = dfs[2].drop(column_to_drop, axis=1)
#drop Home/Away column
column_to_drop = 'Home/Away'
dfs[2] = dfs[2].drop(column_to_drop, axis=1)
#rename column
dfs[2].rename(columns={'OFFENSIVE POINTS 1': 'Offensive_points'}, inplace=True)


#add offensive points to first row
dfs[2] = combine_header_and_first_row(dfs[2], 'Offensive_points')
dfs[2] = set_first_row_as_header(dfs[2])
dfs[2] = split_column(dfs[2], 'Offensive_points_Team Offensive_points_Avg Offensive_points_Oppt Offensive_points_Avg')

#add zeroes for misisng values
dfs[2]['TM GP PF Avg'] = dfs[2]['TM GP PF Avg'].apply(lambda x: ' '.join(str(x).split() + ['0'] * (4 - len(str(x).split()))))

#split column
dfs[2] = split_column(dfs[2], 'TM GP PF Avg')

#rename team column
dfs[2].rename(columns={'TM': 'Team'}, inplace=True)

#move team column to front
dfs[2] = dfs[2][['Team'] + [col for col in dfs[2].columns if col != 'Team']]


dfs[2]

,Team,PA,Avg,Avg,Diff,NaN,PER GAME,Offensive_points_Team,Offensive_points_Avg,Offensive_points_Avg,Offensive_points_Oppt,Offensive_points_Avg,Offensive_points_Avg,GP,PF,Avg,Avg
0,BC,439,24.4,27.5,+3.1,NaN,CFL-Wide: 5.15,470,26.1,22.7,408,26.1,22.7,18,495,24.4,27.5
1,CGY,471,26.2,22.9,-3.3,NaN,Offence 4.52,392,21.8,23.8,428,21.8,23.8,18,412,26.2,22.9
2,EDM,517,28.7,20.4,-8.3,NaN,Return 0.63,340,18.9,26.6,478,18.9,26.6,18,367,28.7,20.4
3,HAM,461,25.6,22.7,-2.9,NaN,- Kick 0.21,377,20.9,23.6,425,20.9,23.6,18,408,25.6,22.7
4,MTL,392,21.8,24.6,+2.8,NaN,- Defence 0.42,370,20.6,21.1,380,20.6,21.1,18,442,21.8,24.6
5,OTT,507,28.2,23.1,-5.1,NaN,NaN,373,20.7,25.4,458,20.7,25.4,18,415,28.2,23.1
6,SSK,551,30.6,21.5,-9.1,NaN,NaN,366,20.3,28.3,509,20.3,28.3,18,387,30.6,21.5
7,TOR,396,22.0,32.8,+10.8,NaN,NaN,527,29.3,20.7,372,29.3,20.7,18,591,22.0,32.8
8,WPG,377,20.9,33.0,+12.1,NaN,NaN,571,31.7,18.2,328,31.7,18.2,18,594,20.9,33.0
9,CFL,"4,111",50.8,50.8,NaN,NaN,NaN,"3,786",46.7,46.7,"3,786",46.7,46.7,81,"4,111",50.8,50.8


In [2321]:
#clean team scoring df


#rename columns
dfs[4].rename(columns={'TEAM TOUCHDOWNS': 'Team_TDs'}, inplace=True)
dfs[4].rename(columns={'TEAM POINTS BY QUARTER:': 'Team_points'}, inplace=True)

#combine header and row names
dfs[4] = combine_header_and_first_row(dfs[4], 'Team_TDs')
dfs[4] = combine_header_and_first_row(dfs[4], 'Team_points')

#set first row as header
dfs[4] = set_first_row_as_header(dfs[4])

#split columns on space
dfs[4] = split_column(dfs[4], 'TM TD C1')
dfs[4] = split_column(dfs[4], 'Team_TDs_Rush Team_TDs_Pass Team_TDs_Int')
dfs[4] = split_column(dfs[4], 'Team_points_Q1 Team_points_Q2 Team_points_Q3 Team_points_Q4 Team_points_OT')

#move TM column to front
dfs[4] = dfs[4][['TM'] + [col for col in dfs[4].columns if col != 'TM']]

#rename team column
dfs[4].rename(columns={'TM': 'Team'}, inplace=True)

#add Team_TDs to relevant columns
dfs[4].rename(columns={'FR': 'TM_TDs_FR'}, inplace=True)
dfs[4].rename(columns={'PR': 'TM_TDs_PR'}, inplace=True)
dfs[4].rename(columns={'KOR': 'TM_TDs_KOR'}, inplace=True)
dfs[4].rename(columns={'FGM': 'TM_TDs_FGM'}, inplace=True)
#drop Tm cat since it is same as TD
column_to_drop = 'Tm'
dfs[4] = dfs[4].drop(column_to_drop, axis=1)

dfs[4]

,Team,NaN,C2,FG,Sgl,S/T,TM_TDs_FR,TM_TDs_PR,TM_TDs_KOR,TM_TDs_FGM,TD,C1,Team_TDs_Rush,Team_TDs_Pass,Team_TDs_Int,Team_points_Q1,Team_points_Q2,Team_points_Q3,Team_points_Q4,Team_points_OT
0,BC,NaN,0,50,7,0,2,0,0,1,49,44,10,35,1,110,155,89,141,0
1,CGY,NaN,2,52,5,1,2,0,0,0,36,29,14,19,1,66,133,101,103,9
2,EDM,NaN,1,23,13,1,0,0,1,0,41,35,15,22,3,82,121,74,90,0
3,HAM,NaN,4,42,6,0,2,0,1,1,40,28,12,23,1,47,134,105,122,0
4,MTL,NaN,2,46,7,0,4,2,0,1,43,35,14,17,5,78,124,97,143,0
5,OTT,NaN,7,42,6,0,1,1,1,0,40,29,22,11,4,76,119,80,126,14
6,SSK,NaN,3,40,9,1,0,3,0,0,37,28,14,20,0,77,101,92,103,14
7,TOR,NaN,4,40,10,1,0,4,0,0,66,55,30,26,6,182,145,125,139,0
8,WPG,NaN,1,46,3,2,1,1,0,0,65,57,20,42,1,123,161,144,149,17
9,CFL,NaN,24,381,66,6,12,11,3,3,417,340,151,215,22,841,"1,193",907,"1,116",54


In [2322]:
#clean oppnent scoring df
#dfs[5]



#rename columns
dfs[5].rename(columns={'OPPONENT TOUCHDOWNS': 'Oppt_TDs'}, inplace=True)
dfs[5].rename(columns={'OPPT POINTS BY QUARTER:': 'Oppt_points'}, inplace=True)

#combine header and row names
dfs[5] = combine_header_and_first_row(dfs[5], 'Oppt_TDs')
dfs[5] = combine_header_and_first_row(dfs[5], 'Oppt_points')

#set first row as header
dfs[5] = set_first_row_as_header(dfs[5])

#split columns on space

dfs[5] = split_column(dfs[5], 'TM TD C1')
dfs[5] = split_column(dfs[5], 'Oppt_TDs_Rush Oppt_TDs_Pass Oppt_TDs_Int Oppt_TDs_FR')
dfs[5] = split_column(dfs[5], 'Oppt_points_Q1 Oppt_points_Q2 Oppt_points_Q3 Oppt_points_Q4 Oppt_points_OT')

#move TM column to front
dfs[5] = dfs[5][['TM'] + [col for col in dfs[5].columns if col != 'TM']]

#rename team column
dfs[5].rename(columns={'TM': 'Team'}, inplace=True)

#add Oppt_TDs to relevant columns
dfs[5].rename(columns={'PR': 'Oppt_TDs_PR'}, inplace=True)
dfs[5].rename(columns={'KOR': 'Oppt_TDs_KOR'}, inplace=True)
dfs[5].rename(columns={'FGM': 'Oppt_TDs_FGM'}, inplace=True)

#drop Opp cat since it is same as TD
column_to_drop = 'Opp'
dfs[5] = dfs[5].drop(column_to_drop, axis=1)


dfs[5]

,Team,C2,FG,Sgl,S/T,Oppt_TDs_PR,Oppt_TDs_KOR,Oppt_TDs_FGM,TD,C1,Oppt_TDs_Rush,Oppt_TDs_Pass,Oppt_TDs_Int,Oppt_TDs_FR,Oppt_points_Q1,Oppt_points_Q2,Oppt_points_Q3,Oppt_points_Q4,Oppt_points_OT
0,BC,1,45,6,0,1,1,0,43,38,18,20,3,0,68,146,108,109,8
1,CGY,3,46,9,0,4,0,0,47,36,14,26,3,0,89,140,91,137,14
2,EDM,4,48,8,1,0,0,1,52,43,21,25,3,2,102,142,149,124,0
3,HAM,2,41,6,3,2,0,0,47,40,19,23,3,0,113,129,81,138,0
4,MTL,1,39,4,0,0,0,1,39,35,20,17,0,1,98,107,78,109,0
5,OTT,3,41,10,0,2,0,1,55,38,17,30,2,3,98,134,131,135,9
6,SSK,2,47,9,2,2,0,0,57,51,20,31,2,2,88,177,115,162,9
7,TOR,3,34,4,0,0,1,0,42,32,7,31,2,1,76,125,69,126,0
8,WPG,5,40,10,0,0,1,0,35,27,15,12,4,3,109,93,85,76,14
9,CFL,24,381,66,6,11,3,3,417,340,151,215,22,12,841,"1,193",907,"1,116",54


In [2323]:
#clean turnover analysis df
#dfs[6]

#drop league wide columns
column_to_drop = 'CFL TURNOVERS'
dfs[6] = dfs[6].drop(column_to_drop, axis=1)


#combine header with first row
dfs[6] = combine_header_and_first_row(dfs[6], 'GIVEAWAYS:')
dfs[6] = combine_header_and_first_row(dfs[6], 'TAKEAWAYS:')
dfs[6] = combine_header_and_first_row(dfs[6], 'Opp')
dfs[6] = combine_header_and_first_row(dfs[6], 'Opp Team')

#drop header
dfs[6] = set_first_row_as_header(dfs[6])

#add giveaways/takeaways to relevant columns
dfs[6].columns.values[3] = 'GIVEAWAYS:_Dns'
dfs[6].rename(columns={'Dns': 'TAKEAWAYS:_Dns'}, inplace=True)

#rename column for split:
dfs[6].rename(columns={'Opp Team_T/O Opp Team_Pts': 'Opp_team_T/O Team_pts'}, inplace=True)

#split columns on space
dfs[6] = split_column(dfs[6], 'Team Ratio')
dfs[6] = split_column(dfs[6], 'GIVEAWAYS:_Fum GIVEAWAYS:_Int')
dfs[6] = split_column(dfs[6], 'TAKEAWAYS:_Fum TAKEAWAYS:_Int')
dfs[6] = split_column(dfs[6], 'Opp_team_T/O Team_pts')

#move team column to front
dfs[6] = dfs[6][['Team'] + [col for col in dfs[6].columns if col != 'Team']]

#drop unneeded columns
column_to_drop = 'RK'
dfs[6] = dfs[6].drop(column_to_drop, axis=1)

#rename columns for clarity
dfs[6].rename(columns={'OFF1': 'Offensive_giveaways'}, inplace=True)
dfs[6].rename(columns={'Ret2': 'Return_giveaways'}, inplace=True)



dfs[6]

,Team,GIVEAWAYS:_Dns,T/Os,Opp_Pts,OFF1 Ret2,TAKEAWAYS:_Dns,Ratio,GIVEAWAYS:_Fum,GIVEAWAYS:_Int,TAKEAWAYS:_Fum,TAKEAWAYS:_Int,Opp_team_T/O,Team_pts
0,BC,11,44,150,40 (3),9,-12,10,23,8,15,32,72
1,CGY,7,30,78,28 (2),12,+9,7,16,11,16,39,99
2,EDM,15,43,95,41 (2),7,-18,9,19,5,13,25,35
3,HAM,12,47,118,45 (2),9,-4,12,23,15,19,43,96
4,MTL,5,34,86,32 (2),12,+14,13,16,14,22,48,128
5,OTT,11,47,92,43 (4),6,-4,18,18,13,24,43,125
6,SSK,16,49,118,46 (3),5,-20,15,18,9,15,29,42
7,TOR,7,27,46,25 (2),12,+27,5,15,15,27,54,151
8,WPG,7,35,85,34 (1),19,+8,13,15,12,12,43,120
9,CFL,91,356,868,334 (21),91,2023,102,163,102,163,356,868


In [2324]:
# clean possesion analysis
# dfs[8]

#update list order
item = dfs.pop(8)

#rename columns
dfs[8].rename(columns={'Own 1-20':'Own_1-20'}, inplace=True)
dfs[8].rename(columns={'Own 21-40':'Own_21-40'}, inplace=True)
dfs[8].rename(columns={'Own 41-54':'Own_41-54'}, inplace=True)
dfs[8].rename(columns={'Opp 55-41':'Opp_55-41'}, inplace=True)
dfs[8].rename(columns={'Opp 40-21':'Opp_40-21'}, inplace=True)
dfs[8].rename(columns={'Opp 20-1':'Opp_20-1'}, inplace=True)

#combine header with first row
dfs[8] = combine_header_and_first_row(dfs[8], 'Own_1-20')
dfs[8] = combine_header_and_first_row(dfs[8], 'Own_21-40')
dfs[8] = combine_header_and_first_row(dfs[8], 'Own_41-54')
dfs[8] = combine_header_and_first_row(dfs[8], 'Opp_55-41')
dfs[8] = combine_header_and_first_row(dfs[8], 'Opp_40-21')
dfs[8] = combine_header_and_first_row(dfs[8], 'Opp_20-1')
dfs[8] = combine_header_and_first_row(dfs[8], '2-&-Outs')

#set first row as header
dfs[8] = set_first_row_as_header(dfs[8])

#drop unneeded row
dfs[8] = dfs[8].drop(dfs[8].index[-1])

#split columns
dfs[8] = split_column(dfs[8], 'Poss TD FG')
dfs[8] = split_column(dfs[8], 'Own_1-20_Drv Own_1-20_TD')
dfs[8] = split_column(dfs[8], 'Own_21-40_Drv Own_21-40_TD')
dfs[8] = split_column(dfs[8], 'Own_41-54_Drv Own_41-54_TD')
dfs[8] = split_column(dfs[8], 'Opp_55-41_Drv Opp_55-41_TD')
dfs[8] = split_column(dfs[8], 'Opp_40-21_Drv Opp_40-21_TD')
dfs[8] = split_column(dfs[8], 'Opp_20-1_Drv Opp_20-1_TD')
dfs[8] = split_column(dfs[8], '2-&-Outs_2&O 2-&-Outs_%')

#Rename TM to Team
dfs[8].rename(columns={'TM': 'Team'}, inplace=True)

dfs[8]

,Team,FGM,T/O,Punt,Oth,TD%,Poss,TD,FG,Own_1-20_Drv,...,Own_41-54_Drv,Own_41-54_TD,Opp_55-41_Drv,Opp_55-41_TD,Opp_40-21_Drv,Opp_40-21_TD,Opp_20-1_Drv,Opp_20-1_TD,2-&-Outs_2&O,2-&-Outs_%
0,BC,3,40,97,12,18%,247,45,50,41,...,34,4,16,3,10,1,2,2,71,29%
1,CGY,8,28,122,8,13%,251,33,52,31,...,40,4,14,5,17,4,6,3,89,35%
2,EDM,6,41,121,10,16%,238,37,23,26,...,41,8,12,2,6,1,0,0,78,33%
3,HAM,6,45,102,13,14%,243,35,42,46,...,40,8,20,4,8,0,2,1,85,35%
4,MTL,14,32,103,13,13%,239,31,46,41,...,54,9,14,2,12,3,2,0,76,32%
5,OTT,7,43,117,17,13%,259,33,42,42,...,42,4,13,5,10,4,4,1,93,36%
6,SSK,8,46,117,12,13%,257,34,40,38,...,47,6,14,1,12,5,1,1,81,32%
7,TOR,3,25,110,15,22%,249,56,40,30,...,54,8,22,7,24,11,3,2,77,31%
8,WPG,5,34,78,16,26%,241,62,46,26,...,43,13,18,5,11,6,1,0,71,29%
9,CFL,60,334,967,116,NaN,"2,224",366,381,321,...,395,64,143,34,110,35,21,10,721,32%


In [2325]:
#clean opponent possesion analysis
#dfs[10]

#rename columns wihtout spaces to make cleaning easier
dfs[10].rename(columns={col: col.replace(' ', '_') for col in dfs[10].columns}, inplace=True)

#combine header with first row
dfs[10] = combine_header_and_first_row(dfs[10], 'Own_1-20')
dfs[10] = combine_header_and_first_row(dfs[10], 'Own_21-40')
dfs[10] = combine_header_and_first_row(dfs[10], 'Own_41-54')
dfs[10] = combine_header_and_first_row(dfs[10], 'Opp_55-41')
dfs[10] = combine_header_and_first_row(dfs[10], 'Opp_40-21')
dfs[10] = combine_header_and_first_row(dfs[10], 'Opp_20-1')
dfs[10] = combine_header_and_first_row(dfs[10], 'Opp_2&O')

#set first row as header
dfs[10] = set_first_row_as_header(dfs[10])

#split columns
dfs[10] = split_column(dfs[10], 'Poss TD FG FGM')
dfs[10] = split_column(dfs[10], 'Own_1-20_Drv Own_1-20_TD')
dfs[10] = split_column(dfs[10], 'Own_21-40_Drv Own_21-40_TD')
dfs[10] = split_column(dfs[10], 'Own_41-54_Drv Own_41-54_TD')
dfs[10] = split_column(dfs[10], 'Opp_55-41_Drv Opp_55-41_TD')
dfs[10] = split_column(dfs[10], 'Opp_40-21_Drv Opp_40-21_TD')
dfs[10] = split_column(dfs[10], 'Opp_20-1_Drv Opp_20-1_TD')
dfs[10] = split_column(dfs[10], 'Opp_2&O_2&O Opp_2&O_%')

#Rename TM to Team
dfs[10].rename(columns={'TM': 'Team'}, inplace=True)

dfs[10]

,Team,T/O,Punt,Oth,TD%,Poss,TD,FG,FGM,Own_1-20_Drv,...,Own_41-54_Drv,Own_41-54_TD,Opp_55-41_Drv,Opp_55-41_TD,Opp_40-21_Drv,Opp_40-21_TD,Opp_20-1_Drv,Opp_20-1_TD,Opp_2&O_2&O,Opp_2&O_%
0,BC,30,115,9,15%,246,38,45,9,31,...,53,8,19,5,9,4,5,2,74,30%
1,CGY,35,104,17,16%,246,40,46,4,38,...,40,7,16,4,7,2,4,2,76,31%
2,EDM,24,96,15,19%,237,46,48,8,41,...,46,12,14,4,13,5,1,1,75,32%
3,HAM,39,101,13,17%,245,42,41,9,30,...,53,7,17,0,18,7,2,1,72,29%
4,MTL,45,107,11,15%,245,37,39,6,38,...,35,6,13,4,12,5,1,0,85,35%
5,OTT,41,109,10,19%,254,47,41,6,38,...,51,10,18,4,13,4,2,0,74,29%
6,SSK,26,111,16,20%,256,51,47,5,33,...,43,6,15,4,16,4,4,2,80,31%
7,TOR,51,114,13,15%,256,38,34,6,43,...,28,6,15,5,9,2,0,0,87,34%
8,WPG,43,110,12,11%,239,27,40,7,31,...,47,2,13,2,13,2,2,2,98,41%
9,CFL,334,967,116,NaN,"2,224",366,381,60,323,...,396,64,140,32,110,35,21,10,721,32%


In [2329]:
#clean time of possesion & field position df
#dfs[11]

#drop league wide columns and Nan

# column_to_drop = 'Unnamed: 3'
# dfs[11] = dfs[11].drop(column_to_drop, axis=1)
# column_to_drop = 'Unnamed: 4'
# dfs[11] = dfs[11].drop(column_to_drop, axis=1)
# column_to_drop = 'Unnamed: 5'
# dfs[11] = dfs[11].drop(column_to_drop, axis=1)
# column_to_drop = 'CFL DRIVE TENDENCIES'
# dfs[11] = dfs[11].drop(column_to_drop, axis=1)


# #rename columns wihtout spaces to make cleaning easier
# dfs[11].rename(columns={col: col.replace(' ', '_') for col in dfs[11].columns}, inplace=True)

# #set first row as header
# dfs[11] = set_first_row_as_header(dfs[11])

# #rename columns for clarity
# dfs[11].rename(columns={'AVG DRIVE START YL': 'Team'}, inplace=True)
# dfs[11].rename(columns={'AVG DRV START': 'Opp'}, inplace=True)
# dfs[11].rename(columns={'Opp End 2': 'Starts_from_opp_end'}, inplace=True)
# dfs[11].rename(columns={'Avg Drv Start': 'Opp'}, inplace=True)

# #combine header with first row
# dfs[11] = combine_header_and_first_row(dfs[11], 'Team')
# dfs[11] = combine_header_and_first_row(dfs[11], 'Opp')
# dfs[11] = combine_header_and_first_row(dfs[11], 'Starts_from_opp_end')

# #set first row as header
# dfs[11] = set_first_row_as_header(dfs[11])

# #rename columns for splitting/clarity
# dfs[11].rename(columns={'Team_Poss Team_FP Team_Yds Team_YdL1': 'Team_Poss Team_FP_Yds Team_YdL'}, inplace=True)
# dfs[11].rename(columns={'Opp_Poss Opp_YdL1': 'Opp_Poss Opp_YdL'}, inplace=True)
# dfs[11].rename(columns={'GP + - Gap': 'Gp Games_+ Games_- avg_Gap'}, inplace=True)
# dfs[11].rename(columns={'Team_Team_Poss Team_Team_FP Team_Team_Yds Team_Team_YdL1': 'Team_Poss Team_FP_Yds Team_YdL'}, inplace=True)


# #split columns
# dfs[11] = split_column(dfs[11], 'Team_Poss Team_FP_Yds Team_YdL')
# dfs[11] = split_column(dfs[11], 'Starts_from_opp_end_No Starts_from_opp_end_TD')
# dfs[11] = split_column(dfs[11], 'Opp_Poss Opp_YdL')
# dfs[11] = split_column(dfs[11], 'Gp Games_+ Games_- avg_Gap')

#drop uneeded columns
column_to_drop = 'Rk'
dfs[11] = dfs[11].drop(column_to_drop, axis=1)

#Rename TM to Team
dfs[11].rename(columns={'TM': 'Team'}, inplace=True)

dfs[11]

,Team,TOP,Yds Diff,Team_Poss,Team_FP_Yds,Team_YdL,Starts_from_opp_end_No,Starts_from_opp_end_TD,Opp_Poss,Opp_YdL
0,BC,29:32,-3.5,247,"8,916",36.1,28,6,246,39.6
1,CGY,29:15,+2.9,251,"9,903",39.5,37,12,246,36.5
2,EDM,28:57,+0.6,238,"8,782",36.9,18,3,237,36.3
3,HAM,31:26,-4.7,243,"8,607",35.4,30,5,245,40.1
4,MTL,29:45,+0.9,239,"8,941",37.4,28,5,245,36.5
5,OTT,30:48,-1.7,259,"9,556",36.9,27,10,254,38.6
6,SSK,28:57,-1.7,257,"9,473",36.9,27,7,256,38.5
7,TOR,28:30,+7.4,249,"10,362",41.6,49,20,256,34.3
8,WPG,32:50,-0.3,241,"9,143",37.9,30,11,239,38.2
9,CFL,NaN,0.0,"2,224","83,683",37.6,274,79,"2,224",37.6


In [2330]:
#clean big play analysis df
#dfs[13]

#drop league wide columns and Nan
column_to_drop = 'CFL BIG PLAYS - HISTORY'
dfs[13] = dfs[13].drop(column_to_drop, axis=1)
column_to_drop = 'Unnamed: 2'
dfs[13] = dfs[13].drop(column_to_drop, axis=1)


#set first row as header
dfs[13] = set_first_row_as_header(dfs[13])

#use bring down header function to clean up column names
dfs[13] = combine_header_and_first_row_with_space(dfs[13], '20+ 30+')


#set first row as header
dfs[13] = set_first_row_as_header(dfs[13])

dfs[13].rename(columns={'Rets Rets Rets':'Punt_Rets K/O_Rets FGM_Rets'}, inplace = True)
dfs[13].rename(columns={'Tot Rush Pass':'Opp_off_Tot Opp_off_20+_Rush Opp_off_30+_Pass'}, inplace = True)
dfs[13].rename(columns={'Rets Rets Ret':'Opp_kick_Punt_Rets Opp_kick_K/O_Rets Opp_kick_FGM_Ret'}, inplace = True)


#drop unneeed rows
dfs[13] = dfs[13].drop([10,11,12,13,14])
#split columns
dfs[13] = split_column(dfs[13], 'Team Total Game')
dfs[13] = split_column(dfs[13], '20+_Rush 30+_Pass')
dfs[13] = split_column(dfs[13], 'Punt_Rets K/O_Rets FGM_Rets')
dfs[13] = split_column(dfs[13], 'Opp_off_Tot Opp_off_20+_Rush Opp_off_30+_Pass')
dfs[13] = split_column(dfs[13], 'Opp_kick_Punt_Rets Opp_kick_K/O_Rets Opp_kick_FGM_Ret')

#rename for clarity
dfs[13].rename(columns={'Game':'Per_Game'}, inplace = True)


#move team column to front
dfs[13] = dfs[13][['Team'] + [col for col in dfs[13].columns if col != 'Team']]

dfs[13]

,Team,NaN,Diff,NaN,Total,Per_Game,20+_Rush,30+_Pass,Punt_Rets,K/O_Rets,FGM_Rets,Opp_off_Tot,Opp_off_20+_Rush,Opp_off_30+_Pass,Opp_kick_Punt_Rets,Opp_kick_K/O_Rets,Opp_kick_FGM_Ret
0,BC,NaN,+2,97.0,41,2.28,5,27,3,3,3,39,7,25,3,4,0
1,CGY,NaN,-9,68.0,43,2.39,8,28,4,2,1,52,14,25,8,2,3
2,EDM,NaN,+6,45.0,55,3.06,17,29,3,5,1,49,10,27,9,2,1
3,HAM,NaN,0,81.0,50,2.78,6,27,4,8,5,50,15,28,3,3,1
4,MTL,NaN,+12,70.0,54,3.00,11,29,8,4,2,42,12,22,3,1,4
5,OTT,NaN,-22,87.0,41,2.28,15,15,8,2,1,63,9,42,8,1,3
6,SSK,NaN,-18,NaN,44,2.44,7,29,4,3,1,62,12,39,6,3,2
7,TOR,NaN,+16,NaN,63,3.50,14,35,13,1,0,47,2,28,6,10,1
8,WPG,NaN,+13,NaN,61,3.39,10,43,3,3,2,48,12,26,4,5,1
9,CFL,NaN,0,NaN,452,5.58,93,262,50,31,16,452,93,262,50,31,16


In [2331]:
#clean Red Zone results df
#dfs[14]

#clean index
item = dfs.pop(14)
#drop uneeded columns
column_to_drop = 'Rd Z.1'
dfs[14] = dfs[14].drop(column_to_drop, axis=1)

#rename columns for cleaning
dfs[14].rename(columns={'Opportunities & TDs:.1': 'Opp'}, inplace=True)
dfs[14].rename(columns={'OPPT OTH (NO TD):': 'Opp_(No_TD)'}, inplace=True)
dfs[14].rename(columns={'Op RZ Pts2': 'Opp_RZ'}, inplace=True)

#combine header and first row names:
dfs[14] = combine_header_and_first_row(dfs[14], 'Opp')
dfs[14] = combine_header_and_first_row(dfs[14], 'Opp_(No_TD)')
dfs[14] = combine_header_and_first_row(dfs[14], 'Opp_RZ')


#set first row as header
dfs[14] = set_first_row_as_header(dfs[14])

#split columns
dfs[14] = split_column(dfs[14], 'Att1 TD % Rnk')
dfs[14] = split_column(dfs[14], 'FG T/O Oth')
dfs[14] = split_column(dfs[14], 'Pts %')
dfs[14] = split_column(dfs[14], 'Opp_Att1 Opp_TD Opp_% Opp_Rnk')
dfs[14] = split_column(dfs[14], 'Opp_(No_TD)_FG Opp_(No_TD)_T/O Opp_(No_TD)_Oth')

#drop uneeded columns
column_to_drop = 'Rnk'
dfs[14] = dfs[14].drop(column_to_drop, axis=1)
#drop uneeded columns
column_to_drop = 'Opp_Rnk'
dfs[14] = dfs[14].drop(column_to_drop, axis=1)

#rename columns for clarity
dfs[14].rename(columns={'Att1': 'Att'}, inplace=True)
dfs[14].rename(columns={'Opp_RZ_Pts Opp_RZ_%': 'Opp_RZ_Pts'}, inplace=True)

dfs[14]


,TEAM,Opp_RZ_Pts,Att,TD,%,FG,T/O,Oth,Pts,%,Opp_Att1,Opp_TD,Opp_%,Opp_(No_TD)_FG,Opp_(No_TD)_T/O,Opp_(No_TD)_Oth
0,BC,270 69%,52,31,59.6,15,5,1,253,70%,56,30,53.6,20,6,0
1,CGY,213 69%,47,24,51.1,18,5,0,221,67%,44,24,54.5,16,4,0
2,EDM,308 70%,36,24,66.7,7,5,0,187,74%,63,34,54.0,23,3,3
3,HAM,266 69%,57,31,54.4,22,4,0,281,70%,55,31,56.4,16,7,1
4,MTL,245 66%,47,23,48.9,16,8,0,208,63%,53,28,52.8,17,8,0
5,OTT,286 72%,50,23,46.0,17,8,2,213,61%,57,34,59.6,20,3,0
6,SSK,281 74%,47,24,51.1,16,7,0,215,65%,54,33,61.1,16,5,0
7,TOR,232 69%,63,40,63.5,18,4,1,335,76%,48,28,58.3,13,6,1
8,WPG,176 63%,71,42,59.2,23,5,1,364,73%,40,20,50.0,11,9,0
9,CFL,"2,277 69%",470,262,55.7,152,51,5,"2,277",69%,470,262,55.7,152,51,5


In [2332]:
#clean team net offence DF
dfs[17]

#drop uneeded columns
column_to_drop = 'CFL.1'
dfs[17] = dfs[17].drop(column_to_drop, axis=1)

#set first row as header
dfs[17] = set_first_row_as_header(dfs[17])

#rename columns for cleaning
dfs[17].rename(columns={'Pass Rush Tm Ls': 'Pass_yds Rush_yds TLs_yds'}, inplace=True)
dfs[17].rename(columns={'Game': 'yds_per_game'}, inplace=True)
dfs[17].rename(columns={'Plys': 'Off_plays'}, inplace=True)
dfs[17].rename(columns={'Per Pl': 'yds_per_play'}, inplace=True)
dfs[17].rename(columns={'Pass SA': 'Pass_plays Sa_Plays'}, inplace=True)
dfs[17].rename(columns={'Rush': 'Rush_plays'}, inplace=True)
dfs[17].rename(columns={'TLs': 'Oth_TLs_Plays'}, inplace=True)
dfs[17].rename(columns={'% Rs %Ps+ Oth': 'Rush% Ps% Oth%'}, inplace=True)

#split columns
dfs[17] = split_column(dfs[17], 'Pass_yds Rush_yds TLs_yds')
dfs[17] = split_column(dfs[17], 'Rush% Ps% Oth%')
dfs[17] = split_column(dfs[17], 'Pass_plays Sa_Plays')

#Rename TM to Team
dfs[17].rename(columns={'TM': 'Team'}, inplace=True)


dfs[17]


,Team,GP,Yards,Per G,Rank,Plays,yds_per_play,Rush_plays,TLs*,Pass_yds,Rush_yds,TLs_yds,Rush%,Ps%,Oth%,Pass_plays,Sa_Plays
0,BC,18,"6,682",371.2,3,985,6.78,297,22,"5,672","1,391",-381,30%,68%,2%,613,53
1,CGY,18,"5,787",321.5,8,980,5.91,346,11,"4,266","1,791",-270,35%,64%,1%,589,34
2,EDM,18,"5,848",324.9,7,895,6.53,367,8,"3,980","2,224",-356,41%,58%,1%,468,52
3,HAM,18,"6,336",352.0,4,993,6.38,346,14,"4,870","1,740",-274,35%,64%,1%,587,46
4,MTL,18,"5,935",329.7,6,953,6.23,354,19,"4,500","1,840",-405,37%,61%,2%,519,61
5,OTT,18,"5,738",318.8,9,"1,017",5.64,411,17,"3,967","2,296",-525,40%,58%,2%,518,71
6,SSK,18,"6,236",346.4,5,"1,035",6.03,342,8,"5,047","1,519",-330,33%,66%,1%,631,54
7,TOR,18,"6,801",377.8,2,926,7.34,389,21,"4,909","2,046",-154,42%,56%,2%,497,19
8,WPG,18,"7,464",414.7,1,"1,031",7.24,448,29,"5,263","2,503",-302,43%,54%,3%,521,33
9,CFL,81,"56,827",701.6,NaN,"8,815",6.45,"3,300",149,"42,474","17,350","-2,997",37%,###,2%,"4,943",423


In [2333]:
#clean oppt net offence df
#dfs[18]

#drop uneeded columns
column_to_drop = 'CFL.1'
dfs[18] = dfs[18].drop(column_to_drop, axis=1)

#set first row as header
dfs[18] = set_first_row_as_header(dfs[18])

#rename columns for cleaning
dfs[18].rename(columns={'Pass Rush Tm L': 'Pass_yds Rush_yds TLs_yds'}, inplace=True)
dfs[18].rename(columns={'Game': 'yds_per_game'}, inplace=True)
dfs[18].rename(columns={'Plys': 'Off_plays'}, inplace=True)
dfs[18].rename(columns={'Per Pl': 'yds_per_play'}, inplace=True)
dfs[18].rename(columns={'Pass SA': 'Pass_plays Sa_Plays'}, inplace=True)
dfs[18].rename(columns={'Rush': 'Rush_plays'}, inplace=True)
dfs[18].rename(columns={'TLs': 'Oth_TLs_Plays'}, inplace=True)
dfs[18].rename(columns={'% Rs%Ps+ Oth': 'Rush% Ps% Oth%'}, inplace=True)

#split columns
dfs[18] = split_column(dfs[18], 'Pass_yds Rush_yds TLs_yds')
dfs[18] = split_column(dfs[18], 'Rush% Ps% Oth%')
dfs[18] = split_column(dfs[18], 'Pass_plays Sa_Plays')

#drop uneeded columns
column_to_drop = 'Rank'
dfs[18] = dfs[18].drop(column_to_drop, axis=1)

#Rename TM to Team
dfs[18].rename(columns={'TM': 'Team'}, inplace=True)


dfs[18]

,Team,GP,Yards,Per G,Plays,yds_per_play,Rush_plays,Oth_TLs_Plays,Pass_yds,Rush_yds,TLs_yds,Rush%,Ps%,Oth%,Pass_plays,Sa_Plays
0,BC,18,"5,918",328.8,992,5.97,346,6,"4,531","1,787",-400,35%,65%,1%,585,55
1,CGY,18,"6,284",349.1,999,6.29,398,31,"4,344","2,246",-306,40%,57%,3%,526,44
2,EDM,18,"6,762",375.7,989,6.84,431,24,"4,531","2,459",-228,44%,54%,2%,495,39
3,HAM,18,"6,445",358.1,966,6.67,379,20,"4,752","2,016",-323,39%,59%,2%,526,41
4,MTL,18,"5,972",331.8,972,6.14,347,16,"4,304","1,978",-310,36%,63%,2%,568,41
5,OTT,18,"6,798",377.7,982,6.92,352,17,"5,591","1,523",-316,36%,62%,2%,568,45
6,SSK,18,"6,864",381.3,980,7.00,413,21,"4,929","2,208",-273,42%,56%,2%,509,37
7,TOR,18,"6,406",355.9,"1,052",6.09,342,4,"5,363","1,505",-462,33%,67%,0%,638,68
8,WPG,18,"5,378",298.8,883,6.09,292,10,"4,129","1,628",-379,33%,66%,1%,528,53
9,CFL,81,"56,827",701.6,"8,815",6.45,"3,300",149,"42,474","17,350","-2,997",37%,###,2%,"4,943",423


In [2334]:
#clean net offence on 1st down df and 1st downs made
dfs[19]


#copy DF so we can turn it into 2 seperate dfs
copied_df = dfs[19].copy()
dfs.insert(20, copied_df)

In [2335]:
#First clean offence on 1st down DF
#dfs[19]


column_to_drop = 'CFL'
dfs[19] = dfs[19].drop(column_to_drop, axis=1)
column_to_drop = 'CFL.1'
dfs[19] = dfs[19].drop(column_to_drop, axis=1)
column_to_drop = 'Unnamed: 1'
dfs[19] = dfs[19].drop(column_to_drop, axis=1)
column_to_drop = '1ST DOWNS - TEAM:'
dfs[19] = dfs[19].drop(column_to_drop, axis=1)
column_to_drop = '1ST DNS - OPPT:'
dfs[19] = dfs[19].drop(column_to_drop, axis=1)

#combine header and first row names (to avoid having columns with matching names):
dfs[19] = combine_header_and_first_row(dfs[19], 'Opp 1st Dn:')

#set first row as header
dfs[19] = set_first_row_as_header(dfs[19])



#rename columns for cleaning and clarity
dfs[19].rename(columns={'Plays Avg Yd': '1st_down_plays 1st_down_avg_yds'}, inplace=True)
dfs[19].rename(columns={'Opp 1st Dn:_Plays Opp 1st Dn:_Av Opp 1st Dn:_Yd': 'Opp_1st_dn_plays Opp_1st_dn_avg_yds'}, inplace=True)
dfs[19].rename(columns={'Rush Pass+': '1st_down_rush_calls 1st_down_pass_calls'}, inplace=True)

#split columns
dfs[19] = split_column(dfs[19], '1st_down_plays 1st_down_avg_yds')
dfs[19] = split_column(dfs[19], 'Opp_1st_dn_plays Opp_1st_dn_avg_yds')
# dfs[19] = split_column(dfs[19], '1st_down_rush_calls 1st_down_pass_calls')



# rename TM to Team
dfs[19].rename(columns={'TM': 'Team'}, inplace=True)


dfs[19]


,Team,Rush Pass+,Team Oppt,1st_down_plays,1st_down_avg_yds,Opp_1st_dn_plays,Opp_1st_dn_avg_yds
0,BC,184 385,"4,055 3,567",569,7.13,556,6.42
1,CGY,228 314,"3,401 3,743",542,6.27,555,6.74
2,EDM,231 281,"3,715 3,777",512,7.26,557,6.78
3,HAM,235 305,"3,217 4,213",540,5.96,565,7.46
4,MTL,228 309,"3,551 3,257",537,6.61,548,5.94
5,OTT,262 317,"3,581 4,101",579,6.18,557,7.36
6,SSK,203 366,"3,305 4,064",569,5.81,563,7.22
7,TOR,267 281,"4,079 3,490",548,7.44,592,5.90
8,WPG,281 314,"4,406 3,098",595,7.41,498,6.22
9,CFL,"2,119 2,872","33,310","4,991",6.67,"4,991",6.67


In [2336]:
#clean 1st downs made
#dfs[20]

#drop unneeded columns
column_to_drop = 'Unnamed: 0'
dfs[20] = dfs[20].drop(column_to_drop, axis=1)
column_to_drop = 'On 1st Down'
dfs[20] = dfs[20].drop(column_to_drop, axis=1)
column_to_drop = '1ST D CALLS'
dfs[20] = dfs[20].drop(column_to_drop, axis=1)
column_to_drop = 'CFL'
dfs[20] = dfs[20].drop(column_to_drop, axis=1)
column_to_drop = 'Opp 1st Dn:'
dfs[20] = dfs[20].drop(column_to_drop, axis=1)
column_to_drop = 'CFL.1'
dfs[20] = dfs[20].drop(column_to_drop, axis=1)
column_to_drop = '1st Dn Yds:'
dfs[20] = dfs[20].drop(column_to_drop, axis=1)

#rename columns for cleaning and clarity
dfs[20].rename(columns={'1ST DOWNS - TEAM:': '1st_downs_made'}, inplace=True)
dfs[20].rename(columns={'1ST DNS - OPPT:': 'Opp_1st_downs_made'}, inplace=True)


#combine header and first row names (to avoid having columns with matching names):
dfs[20] = combine_header_and_first_row(dfs[20], '1st_downs_made')
dfs[20] = combine_header_and_first_row(dfs[20], 'Opp_1st_downs_made')


#set first row as header
dfs[20] = set_first_row_as_header(dfs[20])

#rename columns for cleaning 
dfs[20].rename(columns={'Opp_1st_downs_made_Opp Opp_1st_downs_made_F Opp_1st_downs_made_Rush Opp_1st_downs_made_Pass': 'Opp_1st_downs_made_Tot Opp_1st_downs_made_Rush Opp_1st_downs_made_Pass'}, inplace=True)


#split columns
dfs[20] = split_column(dfs[20], '1st_downs_made_Tot 1st_downs_made_Rush 1st_downs_made_Pass 1st_downs_made_Pen')
dfs[20] = split_column(dfs[20], 'Opp_1st_downs_made_Tot Opp_1st_downs_made_Rush Opp_1st_downs_made_Pass')

#rename columns for clarity
dfs[20].rename(columns={'TM': 'Team'}, inplace=True)


dfs[20]

,Team,1st_downs_made_Tot,1st_downs_made_Rush,1st_downs_made_Pass,1st_downs_made_Pen,Opp_1st_downs_made_Tot,Opp_1st_downs_made_Rush,Opp_1st_downs_made_Pass
0,BC,383,96,260,27,375,130,209
1,CGY,327,118,185,24,361,152,186
2,EDM,323,126,164,33,380,149,197
3,HAM,342,115,205,22,373,127,219
4,MTL,331,111,192,28,347,120,193
5,OTT,360,147,175,38,360,108,228
6,SSK,361,117,223,21,362,136,205
7,TOR,364,132,204,28,376,115,229
8,WPG,423,165,222,36,280,90,164
9,CFL,3214,"1,127","1,830",257,"3,214",1127,1830


In [2337]:
#clean rushing analysis DF
#dfs[21]

#rename columns for cleaning
dfs[21].rename(columns={'1ST DOWN RUSH:2': '1st_down_rush'}, inplace=True)
dfs[21].rename(columns={'QB RUSHING:': 'QB_rush'}, inplace=True)
dfs[21].rename(columns={'OPPONENT RUSHING:': 'Opp_rush'}, inplace=True)
dfs[21].rename(columns={'Unnamed: 4': 'Opp_rush_TD'}, inplace=True)
dfs[21].rename(columns={'Unnamed: 5': 'Opp_rush_10+'}, inplace=True)
dfs[21].rename(columns={'Unnamed: 6': 'Opp_rush_20+'}, inplace=True)
dfs[21].rename(columns={'Unnamed: 3': 'Opp_rush_10+'}, inplace=True)

#combine header with 1st row
dfs[21] = combine_header_and_first_row(dfs[21], '1st_down_rush')
dfs[21] = combine_header_and_first_row(dfs[21], 'QB_rush')
dfs[21] = combine_header_and_first_row(dfs[21], 'Opp_rush')
dfs[21] = combine_header_and_first_row(dfs[21], 'Opp_rush_TD')
dfs[21] = combine_header_and_first_row(dfs[21], 'Opp_rush_10+')

#set first row as header
dfs[21] = set_first_row_as_header(dfs[21])

#rename for cleaning
dfs[21].rename(columns={'1st_down_rush_1st_down_rush_Att 1st_down_rush_1st_down_rush_Yds 1st_down_rush_1st_down_rush_Avg': '1st_down_rush_Att 1st_down_rush_Yds 1st_down_rush_Avg'}, inplace=True)
dfs[21].rename(columns={'QB_rush_QB_rush_Att QB_rush_QB_rush_Yds QB_rush_QB_rush_Esc3': 'QB_rush_Att QB_rush_Yds QB_rush_Esc3'}, inplace=True)
dfs[21].rename(columns={'Opp_rush_Opp_rush_Atts Opp_rush_Opp_rush_Yards Opp_rush_Opp_rush_Avg Opp_rush_Opp_rush_TD': 'Opp_rush_Atts Opp_rush_Yards Opp_rush_Avg Opp_rush_TD'}, inplace=True)
 
# #split columns
dfs[21] = split_column(dfs[21], 'TM Att Yards Avg')
dfs[21] = split_column(dfs[21], '1st_down_rush_Att 1st_down_rush_Yds 1st_down_rush_Avg')
dfs[21] = split_column(dfs[21], 'QB_rush_Att QB_rush_Yds QB_rush_Esc3')
dfs[21] = split_column(dfs[21], 'Opp_rush_Atts Opp_rush_Yards Opp_rush_Avg Opp_rush_TD')

#rename columns for clarity
dfs[21].rename(columns={'PerG': 'yds_per_game'}, inplace=True)
dfs[21].rename(columns={'Opp_rush_10+_10+': 'Opp_rush_10+'}, inplace=True)
dfs[21].rename(columns={'Opp_rush_TD_20+': 'Opp_rush_20+'}, inplace=True)
dfs[21].rename(columns={'QB_rush_Esc3': 'QB_rush_Esc'}, inplace=True)
dfs[21].rename(columns={'TM': 'Team'}, inplace=True)

#move team column to front
dfs[21] = dfs[21][['Team'] + [col for col in dfs[21].columns if col != 'Team']]

dfs[21]

,Team,TD,10+,20+,yds_per_game,Opp_rush_10+,Opp_rush_20+,Att,Yards,Avg,1st_down_rush_Att,1st_down_rush_Yds,1st_down_rush_Avg,QB_rush_Att,QB_rush_Yds,QB_rush_Esc,Opp_rush_Atts,Opp_rush_Yards,Opp_rush_Avg,Opp_rush_TD
0,BC,10,35,5,77.3,45,7,297,"1,391",4.7,185,893,4.8,86,420,37,346,"1,787",5.2,18
1,CGY,14,49,8,99.5,61,14,346,"1,791",5.2,228,"1,206",5.3,70,259,18,398,"2,246",5.6,14
2,EDM,15,73,17,123.6,66,10,367,"2,224",6.1,231,"1,492",6.5,129,878,70,431,"2,459",5.7,21
3,HAM,12,58,6,96.7,57,15,346,"1,740",5.0,235,"1,184",5.0,83,449,35,379,"2,016",5.3,19
4,MTL,14,41,11,102.2,63,12,354,"1,840",5.2,229,"1,227",5.4,106,524,43,347,"1,978",5.7,20
5,OTT,22,77,15,127.6,41,9,411,"2,296",5.6,274,"1,488",5.4,146,914,67,352,"1,523",4.3,17
6,SSK,14,46,7,84.4,70,12,342,"1,519",4.4,203,959,4.7,85,334,29,413,"2,208",5.3,20
7,TOR,30,55,14,113.7,49,2,389,"2,046",5.3,267,"1,385",5.2,75,374,30,342,"1,505",4.4,7
8,WPG,20,66,10,139.1,48,12,448,"2,503",5.6,284,"1,671",5.9,81,279,15,292,"1,628",5.6,15
9,CFL,151,500,93,214.2,500,93,"3,300","17,350",5.26,"2,136","11,505",5.39,861,"4,431",344,"3,300",17350,5.3,151


In [2341]:
#clean passing analysis base data
#dfs[22]

# #set first row as header
# dfs[22] = set_first_row_as_header(dfs[22])


#rename columns for clarity and cleaning
dfs[22].rename(columns={'TM Att Com %': 'TM Att Com complete%'}, inplace=True)
dfs[22].rename(columns={'TM Att Com % Yards': 'TM Att Com complete% Yards'}, inplace=True)
dfs[22].rename(columns={'2D1': '2D_conversions'}, inplace=True)
dfs[22].rename(columns={'Effic IC% Avg YAC TDI': 'Effic IC% Avg YAC TDI'}, inplace=True)
dfs[22].rename(columns={'Effic IC% Avg YAC TDI PerG': 'Effic IC% Avg YAC TDI Yds_per_game'}, inplace=True)
dfs[22].rename(columns={'Per G': 'Yds_per_game'}, inplace=True)
dfs[22].rename(columns={'Depth': 'Avg_depth'}, inplace=True)
dfs[22].rename(columns={'Depth Yds': 'Avg_depth Yds_depth'}, inplace=True)
dfs[22].rename(columns={'%': 'Sack%'}, inplace=True)
dfs[22].rename(columns={'Runs': 'Esc_runs'}, inplace=True)
dfs[22].rename(columns={'Backs': 'Dropbacks'}, inplace=True)
dfs[22].rename(columns={'YAC': '%_YAC'}, inplace=True)

# #drop unneeded rows
# dfs[22] = dfs[22].drop([10, 11, 12, 13, 14])

# #fill NaNs with zeros
# dfs[22] = dfs[22].fillna(0)

# # #fill empty gaps with 0s
# # dfs[22]['TM Att Com complete%'] = dfs[22]['TM Att Com complete%'].apply(lambda x: ' '.join(str(x).split() + ['0'] * (4 - len(str(x).split()))))
# # dfs[22]['Effic IC% Avg YAC TDI'] = dfs[22]['Effic IC% Avg YAC TDI'].apply(lambda x: ' '.join(str(x).split() + ['0'] * (5 - len(str(x).split()))))

# #split columns
dfs[22] = split_column(dfs[22], 'TM Att Com complete%')
dfs[22] = split_column(dfs[22], 'Effic IC% YAC YAC% TDI')

#rename columns for clarity
dfs[22].rename(columns={'TM': 'Team'}, inplace=True)
dfs[22].rename(columns={'PerG': 'Yds_per_game'}, inplace=True)

#move team column to front
dfs[22] = dfs[22][['Team'] + [col for col in dfs[22].columns if col != 'Team']]

dfs[22]

,Team,Yards,Int,TD,LG,30+,2D_conversions,Avg,Yds_per_game,Dropbacks,...,NaN,SA SA%,Att,Com,complete%,Effic,IC%,YAC,YAC%,TDI
0,BC,"5,672",24,35,78,27,122,9.3,315,703,...,0.0,53 8%,613,413,67.4,99.5,3.9,"1,892",33%,1.46
1,CGY,"4,266",16,19,95,28,102,7.2,237,641,...,0.0,34 5%,589,368,62.5,83.8,2.7,"1,841",43%,1.19
2,EDM,"3,980",19,22,102,29,75,8.5,221,590,...,0.0,52 9%,468,299,63.9,89.5,4.1,"1,687",42%,1.16
3,HAM,"4,870",24,23,88,27,131,8.3,271,668,...,0.0,46 7%,587,383,65.2,87.0,4.1,"1,991",41%,0.96
4,MTL,"4,500",16,17,69,29,97,8.7,250,623,...,0.0,61 10%,519,367,70.7,95.2,3.1,"2,163",48%,1.06
5,OTT,"3,967",18,11,55,15,94,7.7,220,656,...,0.0,71 11%,518,345,66.6,82.1,3.5,"1,615",41%,0.61
6,SSK,"5,047",18,20,64,29,110,8.0,280,714,...,0.0,54 8%,631,419,66.4,89.4,2.9,"1,772",35%,1.11
7,TOR,"4,909",15,26,76,35,95,9.9,273,546,...,0.0,19 3%,497,334,67.2,104.1,3.0,"1,926",39%,1.73
8,WPG,"5,263",15,42,71,43,110,10.1,292,569,...,0.0,33 6%,521,357,68.5,116.2,2.9,"1,932",37%,2.80
9,CFL,"42,474",165,215,102,262,936,8.6,524,"5,710",...,0.0,423 7%,"4,943","3,285",66.5,93.9,3.3,"16,819",40%,1.30


In [2349]:
# #clean passing analysis range data
# #dfs[24]




#rename columns for cleaning
dfs[24].rename(columns={'ATTEMPTS 0-9 YDS DEPTH DOWNFIELD:': '0-9_yds'}, inplace=True)
dfs[24].rename(columns={'ATTEMPTS 10-19 YDS DEPTH DOWNFIELD:': '10-19_yds'}, inplace=True)
dfs[24].rename(columns={'ATTEMPTS 20+ YDS DOWNFIELD:': '20+_yds'}, inplace=True)

#combine header with first row
dfs[24] = combine_header_and_first_row(dfs[24], '0-9_yds')
dfs[24] = combine_header_and_first_row(dfs[24], '10-19_yds')
dfs[24] = combine_header_and_first_row(dfs[24], '20+_yds')

#combine unnamed headers with first row to avoid double columns
dfs[24] = combine_header_and_first_row(dfs[24], 'Unnamed: 1')
dfs[24] = combine_header_and_first_row(dfs[24], 'Unnamed: 2')
dfs[24] = combine_header_and_first_row(dfs[24], 'Unnamed: 3')

#fill NaNs with zeros
dfs[24] = dfs[24].fillna(0)

#fill missing values with zeros
dfs[24]['0-9_yds'] = dfs[24]['0-9_yds'].apply(lambda x: ' '.join(str(x).split() + ['0'] * (6 - len(str(x).split()))))
dfs[24]['10-19_yds'] = dfs[24]['10-19_yds'].apply(lambda x: ' '.join(str(x).split() + ['0'] * (7 - len(str(x).split()))))
dfs[24]['20+_yds'] = dfs[24]['20+_yds'].apply(lambda x: ' '.join(str(x).split() + ['0'] * (5 - len(str(x).split()))))

#set first row as header
dfs[24] = set_first_row_as_header(dfs[24])

#split columns
dfs[24] = split_column(dfs[24], '0-9_yds_Att 0-9_yds_Com 0-9_yds_% 0-9_yds_Yds2 0-9_yds_Int 0-9_yds_TD')
dfs[24] = split_column(dfs[24], '10-19_yds_Att 10-19_yds_Com 10-19_yds_% 10-19_yds_Yds2 10-19_yds_Int 10-19_yds_TD 10-19_yds_Effic')
dfs[24] = split_column(dfs[24], '20+_yds_Att 20+_yds_Com 20+_yds_% 20+_yds_Yds2 20+_yds_Int')

#rename columns for clarity
dfs[24].rename(columns={'Unnamed: 1_Effic': '0-9_yds_Effic'}, inplace=True)
dfs[24].rename(columns={'Unnamed: 2_TD': '20+_yds_Effic'}, inplace=True)
dfs[24].rename(columns={'Unnamed: 3_Effic': '20+_yds_Effic'}, inplace=True)
dfs[24].rename(columns={'0-9_yds_Yds2': '0-9_yds_Yds'}, inplace=True)
dfs[24].rename(columns={'10-19_yds_Yds2': '10-19_yds_Yds'}, inplace=True)
dfs[24].rename(columns={'20+_yds_Yds2': '20+_yds_Yds'}, inplace=True)
dfs[24].rename(columns={'TM': 'Team'}, inplace=True)

dfs[24]

,Team,0-9_yds_Effic,20+_yds_Effic,20+_yds_Effic,0-9_yds_Att,0-9_yds_Com,0-9_yds_%,0-9_yds_Yds,0-9_yds_Int,0-9_yds_TD,...,10-19_yds_%,10-19_yds_Yds,10-19_yds_Int,10-19_yds_TD,10-19_yds_Effic,20+_yds_Att,20+_yds_Com,20+_yds_%,20+_yds_Yds,20+_yds_Int
0,BC,106.4,23,115.3,314,258,82.2,"2,323",1,6,...,61.9,"1,555",14,6,70.8,144,59,41.0,"1,794",9
1,CGY,83.8,7,94.4,394,281,71.3,"2,304",6,5,...,51.3,885,6,7,76.0,82,29,35.4,"1,077",4
2,EDM,89.6,11,134.0,300,218,72.7,"1,863",8,11,...,48.0,861,5,0,57.4,70,34,48.6,"1,256",6
3,HAM,91.0,10,81.8,367,277,75.5,"2,411",10,11,...,57.7,"1,113",5,2,79.1,109,42,38.5,"1,346",9
4,MTL,98.5,9,83.7,343,280,81.6,"2,437",5,5,...,55.9,"1,033",2,3,92.5,74,30,40.5,"1,030",9
5,OTT,92.0,3,19.7,345,256,74.2,"2,244",4,6,...,63.7,"1,149",5,2,85.0,60,17,28.3,574,9
6,SSK,88.4,9,68.2,390,297,76.2,"2,363",7,6,...,65.5,"1,606",3,5,107.8,102,31,30.4,"1,078",8
7,TOR,97.2,16,119.9,285,223,78.2,"2,044",4,5,...,62.1,"1,204",3,5,106.6,109,47,43.1,"1,661",8
8,WPG,99.7,21,165.7,290,230,79.3,"2,056",9,13,...,61.0,"1,250",3,8,109.1,113,55,48.7,"1,957",3
9,CFL,93.6,109,102.9,"3,028","2,320",76.6,"20,045",54,68,...,59.0,10656,46,38,87.3,863,344,39.9,"11,773",65


In [2351]:
#clean opponent passing base data DF 
# #dfs[25]



# #set first row as header
# dfs[25] = set_first_row_as_header(dfs[25])

# #rename columns for clarity
# dfs[25].rename(columns={'TM Att Com % Yards': 'TM Att Com Complete% Yards'}, inplace=True)
# dfs[25].rename(columns={'2D1': '2D_conversions'}, inplace=True)
# dfs[25].rename(columns={'Yards': 'yds_per_game'}, inplace=True)
# dfs[25].rename(columns={'Depth': 'Avg_depth'}, inplace=True)
# dfs[25].rename(columns={'Backs SA Runs': 'Dropbacks SA Esc_runs'}, inplace=True)

# #fill NaNs with zeros
# dfs[25] = dfs[25].fillna(0)


# #split columns
# dfs[25] = split_column(dfs[25], 'TM Att Com Complete% Yards')
# dfs[25] = split_column(dfs[25], 'Effic IC% Avg YAC')


# #rename columns for clarity
# dfs[25].rename(columns = {'Backs':'Dropbacks'}, inplace=True)
# dfs[25].rename(columns={'TM': 'Team'}, inplace=True)

# #move team column to front
# dfs[25] = dfs[25][['Team'] + [col for col in dfs[25].columns if col != 'Team']]

dfs[25]

""


In [2352]:
#clean opponnents passing range data
#dfs[26]

empty_df = pd.DataFrame()

# Specify the index where you want to add the empty DataFrame
index_to_insert = 26

# Insert the empty DataFrame into the list at the specified index
dfs.insert(index_to_insert, empty_df)

# #rename columns for cleaning
# dfs[26].rename(columns={'OPPT ATTS 0-9 YDS DEPTH DOWNFIELD: 2': '0-9_yds'}, inplace=True)
# dfs[26].rename(columns={'OPPT ATTS 10-19 YDS DEPTH DOWNFIELD: 2': '10-19_yds'}, inplace=True)
# dfs[26].rename(columns={'OPPONENT ATTS 20+ YDS DOWNFIELD: 2': '20+_yds'}, inplace=True)

# #combine header with first row
# dfs[26] = combine_header_and_first_row(dfs[26], '0-9_yds')
# dfs[26] = combine_header_and_first_row(dfs[26], '10-19_yds')
# dfs[26] = combine_header_and_first_row(dfs[26], '20+_yds')

# #set first row as header
# dfs[26] = set_first_row_as_header(dfs[26])

# #rename columns for cleaning
# dfs[26].rename(columns={'20+_yds_Att 20+_yds_Com 20+_yds_%': '20+_yds_Att 20+_yds_Com 20+_yds_% 20+_yds_Yds'}, inplace=True)
# dfs[26].rename(columns={'20+_yds_Att 20+_yds_Com 20+_yds_% 20+_yds_Yds2 20+_yds_Int 20+_yds_TD': '20+_yds_Att 20+_yds_Com 20+_yds_% 20+_yds_Yds 20+_yds_Int 20+-yds_TD'}, inplace=True)
# dfs[26].rename(columns={'Effic': '20+_yds_Effic'}, inplace=True)
# dfs[26].rename(columns={'0-9_yds_2TM 0-9_yds_Att 0-9_yds_Com 0-9_yds_% 0-9_yds_Yds2 0-9_yds_Int 0-9_yds_TD 0-9_yds_Effic':'Team 0-9_yds_Att 0-9_yds_Com 0-9_yds_% 0-9_yds_Yds 0-9_yds_Int 0-9_yds_TD 0-9_yds_Effic'}, inplace=True)
# dfs[26].rename(columns={'0-9_yds_2TM 0-9_yds_Att 0-9_yds_Com 0-9_yds_% 0-9_yds_Yds 0-9_yds_2 0-9_yds_Int 0-9_yds_TD 0-9_yds_Effic':'Team 0-9_yds_Att 0-9_yds_Com 0-9_yds_% 0-9_yds_Yds 0-9_yds_Int 0-9_yds_TD 0-9_yds_Effic'}, inplace=True)
# dfs[26].rename(columns={'10-19_yds_Att 10-19_yds_Com 10-19_yds_% 10-19_yds_Yds 10-19_yds_2 10-19_yds_Int 10-19_yds_TD 10-19_yds_Effic':'10-19_yds_Att 10-19_yds_Com 10-19_yds_% 10-19_yds_Yds 10-19_yds_Int 10-19_yds_TD 10-19_yds_Effic'}, inplace=True)
# dfs[26].rename(columns={'Int TD Effic':'20+_yds_Int 20+_yds_TD 20+_yds_Effic'}, inplace=True)
# dfs[26].rename(columns={'0-9_yds_Att 0-9_yds_Com 0-9_yds_% 0-9_yds_Yds2 0-9_yds_2Int 0-9_yds_TD 0-9_yds_Effic':'0-9_yds_Att 0-9_yds_Com 0-9_yds_% 0-9_yds_Yds 0-9_yds_Int 0-9_yds_TD 0-9_yds_Effic'}, inplace=True)

# #split columns
# dfs[26] = split_column(dfs[26], '0-9_yds_Att 0-9_yds_Com 0-9_yds_% 0-9_yds_Yds 0-9_yds_Int 0-9_yds_TD 0-9_yds_Effic')
# dfs[26] = split_column(dfs[26], '10-19_yds_Att 10-19_yds_Com 10-19_yds_% 10-19_yds_Yds 10-19_yds_Int 10-19_yds_TD 10-19_yds_Effic')
# dfs[26] = split_column(dfs[26], '20+_yds_Att 20+_yds_Com 20+_yds_% 20+_yds_Yds')
# dfs[26] = split_column(dfs[26], '20+_yds_Int 20+_yds_TD 20+_yds_Effic')


# #rename columns for clarity
# dfs[26].rename(columns={'0-9_yds_Yds2': '0-9_yds_Yds'}, inplace=True)
# dfs[26].rename(columns={'10-19_yds_Yds2': '10-19_yds_Yds'}, inplace=True)
# dfs[26].rename(columns={'0-9_yds_Yds2TM': 'Team'}, inplace=True)
# dfs[26].rename(columns={'TM': 'Team'}, inplace=True)

dfs[26]


""


In [2358]:
#clean 2nd down conversion df
#dfs[27]

#drop uneeded columns
column_to_drop = 'Unnamed: 0'
dfs[27] = dfs[27].drop(column_to_drop, axis=1)
column_to_drop = 'Unnamed: 4'
dfs[27] = dfs[27].drop(column_to_drop, axis=1)


#rename columns for cleaning
dfs[27].rename(columns={'1-3 YARDS:': '1-3_yds'}, inplace=True)
dfs[27].rename(columns={'4-6 YARDS:': '4-6_yds'}, inplace=True)
dfs[27].rename(columns={'7+ YARDS:': '7+_yds'}, inplace=True)
dfs[27].rename(columns={'YARDS TO GO1': 'Yds_to_go'}, inplace=True)

#combine header with first row
dfs[27] = combine_header_and_first_row(dfs[27], '1-3_yds')
dfs[27] = combine_header_and_first_row(dfs[27], 'Unnamed: 1')
dfs[27] = combine_header_and_first_row(dfs[27], '4-6_yds')
dfs[27] = combine_header_and_first_row(dfs[27], 'Unnamed: 2')
dfs[27] = combine_header_and_first_row(dfs[27], '7+_yds')
dfs[27] = combine_header_and_first_row(dfs[27], 'Unnamed: 3')
dfs[27] = combine_header_and_first_row(dfs[27], 'Yds_to_go')

#set first row as header
dfs[27] = set_first_row_as_header(dfs[27])

#split columns
dfs[27] = split_column(dfs[27], 'TM Att / Md %')
dfs[27] = split_column(dfs[27], '1-3_yds_Att 1-3_yds_/ 1-3_yds_Md')
dfs[27] = split_column(dfs[27], '4-6_yds_Att 4-6_yds_/ 4-6_yds_Md')
dfs[27] = split_column(dfs[27], '7+_yds_Att 7+_yds_/ 7+_yds_Md')
dfs[27] = split_column(dfs[27], 'Yds_to_go_Yards Yds_to_go_Avg')

#drop unneeded columns
column_to_drop = '/'
dfs[27] = dfs[27].drop(column_to_drop, axis=1)
column_to_drop = '1-3_yds_/'
dfs[27] = dfs[27].drop(column_to_drop, axis=1)
column_to_drop = '4-6_yds_/'
dfs[27] = dfs[27].drop(column_to_drop, axis=1)
column_to_drop = '7+_yds_/'
dfs[27] = dfs[27].drop(column_to_drop, axis=1)


#rename columns for clarity
dfs[27].rename(columns={'Unnamed: 1_%': '1-3_yds_%'}, inplace=True)
dfs[27].rename(columns={'Unnamed: 2_%': '4-6_yds_%'}, inplace=True)
dfs[27].rename(columns={'Unnamed: 3_%': '7+_yds_%'}, inplace=True)
dfs[27].rename(columns={'TM': 'Team'}, inplace=True)

#move team column to front
dfs[27] = dfs[27][['Team'] + [col for col in dfs[27].columns if col != 'Team']]

dfs[27]

,Team,1-3_yds_%,4-6_yds_%,7+_yds_%,Att,Md,%,1-3_yds_Att,1-3_yds_Md,4-6_yds_Att,4-6_yds_Md,7+_yds_Att,7+_yds_Md,Yds_to_go_Yards,Yds_to_go_Avg
0,BC,73.1,63.8,38.4,380,193,50.8,67,49,94,60,219,84,"2,972",7.82
1,CGY,78.0,40.4,32.3,401,175,43.6,82,64,99,40,220,71,"2,986",7.45
2,EDM,68.1,53.2,32.6,350,159,45.4,72,49,94,50,184,60,"2,619",7.48
3,HAM,67.2,54.1,41.5,410,199,48.5,64,43,98,53,248,103,"3,170",7.73
4,MTL,71.8,46.4,34.8,382,173,45.3,78,56,97,45,207,72,"2,778",7.27
5,OTT,71.8,48.9,33.3,397,176,44.3,78,56,88,43,231,77,"2,918",7.35
6,SSK,66.7,47.2,36.8,408,181,44.4,72,48,89,42,247,91,"3,143",7.70
7,TOR,76.7,52.0,37.9,366,186,50.8,86,66,98,51,182,69,"2,641",7.22
8,WPG,81.6,59.8,41.1,400,227,56.8,103,84,112,67,185,76,"2,532",6.33
9,CFL,73.4,51.9,36.6,"3,494","1,669",47.8,702,515,869,451,"1,923",703,"25,759",7.37


In [2359]:
## clean opponent 2nd down conversion DF
#dfs[28]

#drop uneeded columns
column_to_drop = 'Unnamed: 0'
dfs[28] = dfs[28].drop(column_to_drop, axis=1)
column_to_drop = 'Unnamed: 1'
dfs[28] = dfs[28].drop(column_to_drop, axis=1)

#rename columns for cleaning
dfs[28].rename(columns={'OPPT 1-3 YARDS:': '1-3_yds'}, inplace=True)
dfs[28].rename(columns={'OPPT 4-6 YARDS:': '4-6_yds'}, inplace=True)
dfs[28].rename(columns={'OPPT 7+ YARDS:': '7+_yds'}, inplace=True)
dfs[28].rename(columns={'OPP YARDS TO GO1': 'Yds_to_go'}, inplace=True)

#combine header with first row
dfs[28] = combine_header_and_first_row(dfs[28], '1-3_yds')
dfs[28] = combine_header_and_first_row(dfs[28], '4-6_yds')
dfs[28] = combine_header_and_first_row(dfs[28], '7+_yds')
dfs[28] = combine_header_and_first_row(dfs[28], 'Yds_to_go')

#set first row as header
dfs[28] = set_first_row_as_header(dfs[28])

#split columns
dfs[28] = split_column(dfs[28], 'TM Att / Md %')
dfs[28] = split_column(dfs[28], '1-3_yds_Att 1-3_yds_/ 1-3_yds_Md 1-3_yds_%')
dfs[28] = split_column(dfs[28], '4-6_yds_Att 4-6_yds_/ 4-6_yds_Md 4-6_yds_%')
dfs[28] = split_column(dfs[28], '7+_yds_Att 7+_yds_/ 7+_yds_Md 7+_yds_%')
dfs[28] = split_column(dfs[28], 'Yds_to_go_Yards Yds_to_go_Avg')

#drop unneeded columns
column_to_drop = '/'
dfs[28] = dfs[28].drop(column_to_drop, axis=1)
column_to_drop = '1-3_yds_/'
dfs[28] = dfs[28].drop(column_to_drop, axis=1)
column_to_drop = '4-6_yds_/'
dfs[28] = dfs[28].drop(column_to_drop, axis=1)
column_to_drop = '7+_yds_/'
dfs[28] = dfs[28].drop(column_to_drop, axis=1)
column_to_drop = 'Rk'
dfs[28] = dfs[28].drop(column_to_drop, axis=1)

#rename columns for clarity
dfs[28].rename(columns={'TM': 'Team'}, inplace=True)

dfs[28]

,NaN,Team,Att,Md,%,1-3_yds_Att,1-3_yds_Md,1-3_yds_%,4-6_yds_Att,4-6_yds_Md,4-6_yds_%,7+_yds_Att,7+_yds_Md,7+_yds_%,Yds_to_go_Yards,Yds_to_go_Avg
0,NaN,BC,395,182,46.1,76,57,75.0,100,50,50.0,219,75,34.2,"2,989",7.57
1,NaN,CGY,392,189,48.2,95,74,77.9,106,59,55.7,191,56,29.3,"2,787",7.11
2,NaN,EDM,400,211,52.8,84,65,77.4,109,65,59.6,207,81,39.1,"2,745",6.86
3,NaN,HAM,366,178,48.6,78,56,71.8,83,44,53.0,205,78,38.0,"2,645",7.23
4,NaN,MTL,396,188,47.5,82,58,70.7,101,52,51.5,213,78,36.6,"2,839",7.17
5,NaN,OTT,387,190,49.1,73,51,69.9,98,46,46.9,216,93,43.1,"2,897",7.49
6,NaN,SSK,385,189,49.1,78,61,78.2,95,49,51.6,212,79,37.3,"2,818",7.32
7,NaN,TOR,417,194,46.5,85,56,65.9,84,45,53.6,248,93,37.5,"3,234",7.76
8,NaN,WPG,356,148,41.6,51,37,72.5,93,41,44.1,212,70,33.0,"2,805",7.88
9,NaN,CFL,"3,494","1,669",47.8,702,515,73.4,869,451,51.9,"1,923",703,36.6,"25,759",7.37


In [2360]:
#clean 3rd and short df
#dfs[30]

#drop unneeded columns

column_to_drop = '3RD & SHORT YARDS - HISTORY'
dfs[30] = dfs[30].drop(column_to_drop, axis=1)
column_to_drop = 'Unnamed: 1'
dfs[30] = dfs[30].drop(column_to_drop, axis=1)
column_to_drop = 'Unnamed: 2'
dfs[30] = dfs[30].drop(column_to_drop, axis=1)
column_to_drop = 'Unnamed: 3'
dfs[30] = dfs[30].drop(column_to_drop, axis=1)
column_to_drop = 'Unnamed: 4'
dfs[30] = dfs[30].drop(column_to_drop, axis=1)


#rename columns for cleaning
dfs[30].rename(columns={'BY OPPONENTS:': 'Opp'}, inplace=True)

#combine header with first row
dfs[30] = combine_header_and_first_row(dfs[30], 'Opp')

#set first row as header
dfs[30] = set_first_row_as_header(dfs[30])

#drop unneeded row
dfs[30] = dfs[30].drop(dfs[30].index[-1])




#rename columns for cleaning
dfs[30].rename(columns={'Fail': 'Opp_Fail'}, inplace=True)
dfs[30].rename(columns={'Opp FD Rush Pass': 'Opp_Tot Opp_Rush Opp_Pass'}, inplace=True)

# #split columns
# dfs[30] = split_column(dfs[30], 'Att / Md Fail')
# dfs[30] = split_column(dfs[30], 'Opp_Att Opp_/ Opp_Md')

# #drop unneeded columns
# column_to_drop = '/'
# dfs[30] = dfs[30].drop(column_to_drop, axis=1)
# column_to_drop = 'Opp_/'
# dfs[30] = dfs[30].drop(column_to_drop, axis=1)

#rename for clarity
dfs[30].rename(columns={'TEAM Att / Md': 'Team'}, inplace=True)

dfs[30]

,TM,Att / Md Fail,Opp_Att Opp_/ Opp_Md Opp_%,Team
0,BC,20 / 15 5,25 / 21 84%,BC 81 / 69
1,CGY,27 / 24 3,26 / 18 69%,CGY 96 / 81
2,EDM,22 / 17 5,26 / 22 85%,EDM 109 / 87
3,HAM,23 / 20 3,26 / 22 85%,HAM 93 / 76
4,MTL,22 / 19 3,23 / 19 83%,MTL 103 / 89
5,OTT,27 / 22 5,20 / 16 80%,OTT 93 / 73
6,SSK,39 / 31 8,17 / 14 82%,SSK 119 / 97
7,TOR,11 / 6 5,29 / 26 90%,TOR 82 / 63
8,WPG,26 / 21 5,25 / 17 68%,WPG 104 / 94
9,CFL,217 / 175 42,217 / 175 81%,CFL 880 / 729


In [2145]:
# #clean penalties df
# # dfs[33]

 
# empty_df = pd.DataFrame()

# # Specify the index where you want to add the empty DataFrame
# index_to_insert = 33

# # Insert the empty DataFrame into the list at the specified index

# dfs.insert(index_to_insert, empty_df)


# #rename columns for cleaning
# dfs[33].rename(columns={'ST COVER:': 'ST_cover'}, inplace=True)
# dfs[33].rename(columns={'ST RETURN:': 'ST_return'}, inplace=True)

# #combine header with first row
# dfs[33] = combine_header_and_first_row(dfs[33], 'ST_cover')
# dfs[33] = combine_header_and_first_row(dfs[33], 'ST_return')

# #set first row as header
# dfs[33] = set_first_row_as_header(dfs[33])

# #rename for cleaning
# dfs[33].rename(columns={'ST_return_Pt ST_return_R ST_return_KOR': 'ST_return_PtR ST_return_KOR'}, inplace=True)

# #drop unneeded rows
# dfs[33] = dfs[33].drop(dfs[33].index[-3:])

# #add zeros for missing values
# dfs[33]['ALL Avg'] = dfs[33]['ALL Avg'].apply(lambda x: ' '.join(str(x).split() + ['0'] * (2 - len(str(x).split()))))

# #split columns
# dfs[33] = split_column(dfs[33], 'TM GP')
# dfs[33] = split_column(dfs[33], 'ALL Avg')
# dfs[33] = split_column(dfs[33], 'Off Def SpTm')
# dfs[33] = split_column(dfs[33], 'ST_cover_Punt ST_cover_K/Os')
# dfs[33] = split_column(dfs[33], 'ST_return_PtR ST_return_KOR')

# #rename for clarity
# dfs[33].rename(columns={'TM': 'Team'}, inplace=True)

# #move team column to front
# dfs[33] = dfs[33][['Team'] + [col for col in dfs[33].columns if col != 'Team']]






NameError: name 'list_of_dfs' is not defined

In [2361]:
item = dfs.pop(34)
dfs[34]


,Unnamed: 0,FIELD GOAL ATTEMPTS:,Unnamed: 1,Unnamed: 2,<40 Yard Atts,40+ yard Atts,50+ FGA*,FG Made Yds,FG Att Yds
0,TM,FGA / Md %,LG,S,FGA Md %,FGA Md %,FGA Md,Yards Avg,Yards Avg
1,BC,53 / 5094.3,51,0,25 25 100.0,28 25 89.3,4 2,"1,753 35.1","1,904 35.9"
2,CGY,60 / 5286.7,53,0,30 29 96.7,30 23 76.7,7 6,"1,866 35.9","2,232 37.2"
3,EDM,29 / 2379.3,46,3,14 13 92.9,15 11 73.3,1 0,748 32.5,"1,005 34.7"
4,HAM,48 / 4287.5,50,3,38 37 97.4,10 5 50.0,3 1,"1,249 29.7","1,529 31.9"
5,MTL,61 / 4675.4,51,5,32 28 87.5,29 18 62.1,6 4,"1,538 33.4","2,194 36.0"
6,OTT,49 / 4285.7,54,2,26 25 96.2,23 17 73.9,11 8,"1,441 34.3","1,763 36.0"
7,SSK,48 / 4083.3,54,2,31 28 90.3,17 12 70.6,7 3,"1,292 32.3","1,642 34.2"
8,TOR,43 / 4093.0,53,1,31 31 100.0,12 9 75.0,3 2,"1,267 31.7","1,400 32.6"
9,WPG,51 / 4690.2,53,0,31 32 103.2,19 15 78.9,6 4,"1,477 32.1","1,722 33.8"


In [2362]:
#clean field goals df
#dfs[34]






#rename for cleaning
dfs[34].rename(columns={'<40 Yard Atts':'<40_yds'}, inplace=True)
dfs[34].rename(columns={'40+ yard Atts':'40+_yds'}, inplace=True)
dfs[34].rename(columns={'50+ FGA*':'50+_yds'}, inplace=True)
dfs[34].rename(columns={'FG Made Yds':'FG_made'}, inplace=True)
dfs[34].rename(columns={'FG Att Yds':'FG_att'}, inplace=True)

#combine header with first row
dfs[34] = combine_header_and_first_row(dfs[34], '<40_yds')
dfs[34] = combine_header_and_first_row(dfs[34], '40+_yds')
dfs[34] = combine_header_and_first_row(dfs[34], '50+_yds')
dfs[34] = combine_header_and_first_row(dfs[34], 'FG_made')
dfs[34] = combine_header_and_first_row(dfs[34], 'FG_att')

#set first row as header
dfs[34] = set_first_row_as_header(dfs[34])

#numbers are joined together in DF
#create function to help solve this issue
def modify_column(df, column_name):
    # Define a helper function to modify the string
    def modify_string(s):
        words = s.split()
        last_word = words[-1]
        
        # Check if the last word has 4 characters
        if len(last_word) == 5 or 6:
            modified_word = last_word[0] + ' ' + last_word[1:]
        else:
            # If it doesn't meet the conditions, return the string as is
            return s

        # Reconstruct the string with the modified word
        return ' '.join(words[:-1] + [modified_word])

    # Apply the modification to the specified column
    df[column_name] = df[column_name].apply(modify_string)

    return df

dfs[34] = modify_column(dfs[34], 'FGA / Md %')

#add zeroes for missing values
dfs[34]['FGA / Md %'] = dfs[34]['FGA / Md %'].apply(lambda x: ' '.join(str(x).split() + ['0'] * (4 - len(str(x).split()))))
dfs[34]['<40_yds_FGA <40_yds_Md <40_yds_%'] = dfs[34]['<40_yds_FGA <40_yds_Md <40_yds_%'].apply(lambda x: ' '.join(str(x).split() + ['0'] * (3 - len(str(x).split()))))
dfs[34]['40+_yds_FGA 40+_yds_Md 40+_yds_%'] = dfs[34]['40+_yds_FGA 40+_yds_Md 40+_yds_%'].apply(lambda x: ' '.join(str(x).split() + ['0'] * (3 - len(str(x).split()))))
dfs[34]['FG_made_Yards FG_made_Avg'] = dfs[34]['FG_made_Yards FG_made_Avg'].apply(lambda x: ' '.join(str(x).split() + ['0'] * (2 - len(str(x).split()))))
dfs[34]['FG_att_Yards FG_att_Avg'] = dfs[34]['FG_att_Yards FG_att_Avg'].apply(lambda x: ' '.join(str(x).split() + ['0'] * (2 - len(str(x).split()))))

#split columns
dfs[34] = split_column(dfs[34], 'FGA / Md %')
dfs[34] = split_column(dfs[34], '<40_yds_FGA <40_yds_Md <40_yds_%')
dfs[34] = split_column(dfs[34], '40+_yds_FGA 40+_yds_Md 40+_yds_%')
dfs[34] = split_column(dfs[34], '50+_yds_FGA 50+_yds_Md')
dfs[34] = split_column(dfs[34], 'FG_made_Yards FG_made_Avg')
dfs[34] = split_column(dfs[34], 'FG_att_Yards FG_att_Avg')

#remove unneeded columns
column_to_drop = '/'
dfs[34] = dfs[34].drop(column_to_drop, axis=1)

#rename for clarity
dfs[34].rename(columns={'TM': 'Team'}, inplace=True)

dfs[34]


/var/folders/pg/qyrtrfln0b9761tg47bv7rhc0000gn/T/ipykernel_88897/1333315924.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = df[column_name].apply(modify_string)
/var/folders/pg/qyrtrfln0b9761tg47bv7rhc0000gn/T/ipykernel_88897/1333315924.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs[34]['FGA / Md %'] = dfs[34]['FGA / Md %'].apply(lambda x: ' '.join(str(x).split() + ['0'] * (4 - len(str(x).split()))))
/var/folders/pg/qyrtrfln0b9761tg47bv7rhc0000gn/T/ipykernel_88897/1333315924

,Team,LG,S,FGA,Md,%,<40_yds_FGA,<40_yds_Md,<40_yds_%,40+_yds_FGA,40+_yds_Md,40+_yds_%,50+_yds_FGA,50+_yds_Md,FG_made_Yards,FG_made_Avg,FG_att_Yards,FG_att_Avg
0,BC,51,0,53,5,094.3,25,25,100.0,28,25,89.3,4,2,"1,753",35.1,"1,904",35.9
1,CGY,53,0,60,5,286.7,30,29,96.7,30,23,76.7,7,6,"1,866",35.9,"2,232",37.2
2,EDM,46,3,29,2,379.3,14,13,92.9,15,11,73.3,1,0,748,32.5,"1,005",34.7
3,HAM,50,3,48,4,287.5,38,37,97.4,10,5,50.0,3,1,"1,249",29.7,"1,529",31.9
4,MTL,51,5,61,4,675.4,32,28,87.5,29,18,62.1,6,4,"1,538",33.4,"2,194",36.0
5,OTT,54,2,49,4,285.7,26,25,96.2,23,17,73.9,11,8,"1,441",34.3,"1,763",36.0
6,SSK,54,2,48,4,083.3,31,28,90.3,17,12,70.6,7,3,"1,292",32.3,"1,642",34.2
7,TOR,53,1,43,4,093.0,31,31,100.0,12,9,75.0,3,2,"1,267",31.7,"1,400",32.6
8,WPG,53,0,51,4,690.2,31,32,103.2,19,15,78.9,6,4,"1,477",32.1,"1,722",33.8
9,CFL,54,16,442,3,8186.2,258,248,96.1,183,135,73.8,48,30,"12,631",33.2,"15,391",34.8


In [2363]:
#clean converts df
#dfs[36]

#pop unneeded df
item = dfs.pop(36)

#set first row as header
dfs[36] = set_first_row_as_header(dfs[36])

#rename columns for cleaning
dfs[36].rename(columns={'Att / C-1 %': 'C1_att C1_/ C1_md C1_%'}, inplace=True)
dfs[36].rename(columns={'Miss': 'C1_Miss'}, inplace=True)
dfs[36].rename(columns={'Att / C-1 % Miss':'C1_att C1_/ C1_md C1_% C1_miss'}, inplace=True)
dfs[36].rename(columns={'Atts / C-2 % Md': 'C2_att C2_/ C2_md C2_%'}, inplace=True)
dfs[36].rename(columns={'Rsh': 'C2_Rsh'}, inplace=True)
dfs[36].rename(columns={'Pass':'C2_Pass'}, inplace=True)
dfs[36].rename(columns={'Def':'C2_Def'}, inplace=True)
dfs[36].rename(columns={'Rsh Pass':'Rsh_att Pass_att'}, inplace=True)
dfs[36].rename(columns={'Atts / C-2':'C2_att C2_/ C2_md'}, inplace=True)


# #add zeroes for missing values
# dfs[36]['C2_att C2_/ C2_md C2_%'] = dfs[36]['C2_att C2_/ C2_md C2_%'].apply(lambda x: ' '.join(str(x).split() + ['0'] * (4 - len(str(x).split()))))

#drop column
column_to_drop = 'Year Att Md %'
dfs[36] = dfs[36].drop(column_to_drop, axis=1)

#split columns
# dfs[36] = split_column(dfs[36], 'Rsh_att Pass_att')
dfs[36] = split_column(dfs[36], 'C1_att C1_/ C1_md C1_% C1_miss')
dfs[36] = split_column(dfs[36], 'C2_att C2_/ C2_md')
#drop unneeded columns
column_to_drop = 'C1_/'
dfs[36] = dfs[36].drop(column_to_drop, axis=1)
column_to_drop = 'C2_/'
dfs[36] = dfs[36].drop(column_to_drop, axis=1)

#rename columns for clarity
dfs[36].rename(columns={'TM': 'Team'}, inplace=True)

dfs[36]

,Team,NaN,Team,% Md,Rsh_att Pass_att,C2_Def,Rsh_att Pass_att,C1_att,C1_md,C1_%,C1_miss,C2_att,C2_md
0,BC,NaN,BC,0.0,0 0,0,1 3,45,44,97.8,1,4,0
1,CGY,NaN,CGY,33.3,0 2,0,0 6,30,29,96.7,1,6,2
2,EDM,NaN,EDM,25.0,1 0,0,2 2,37,35,94.6,2,4,1
3,HAM,NaN,HAM,50.0,0 3,1,2 5,33,28,84.8,5,8,4
4,MTL,NaN,MTL,33.3,0 2,0,0 6,37,35,94.6,2,6,2
5,OTT,NaN,OTT,70.0,1 6,0,1 9,29,29,100.0,0,10,7
6,SSK,NaN,SSK,42.9,0 3,0,0 7,30,28,93.3,2,7,3
7,TOR,NaN,TOR,66.7,2 2,0,3 3,60,55,91.7,5,6,4
8,WPG,NaN,WPG,33.3,0 1,0,0 3,62,57,91.9,5,3,1
9,CFL,NaN,CFL,44.4,4 19,1,9 44,363,340,93.7,23,54,24


In [2364]:
#clean kick return df
#dfs[37]

index1 = 37
index2 = 41

# Ensure indexes are within the range of the list
if index1 < len(dfs) and index2 < len(dfs):
    # Swapping process
    temp = dfs[index1]  # Store the first item temporarily
    dfs[index1] = dfs[index2]  # Overwrite the first item with the second
    dfs[index2] = temp  # Assign the stored item to the second position
else:
    print("Error: Indexes out of range")



#drop unneeded columns
column_to_drop = 'CFL'
dfs[37] = dfs[37].drop(column_to_drop, axis=1)


#rename columns for cleaning
dfs[37].rename(columns={'TEAM PUNT RETURNS:': 'Punt_returns'}, inplace=True)
dfs[37].rename(columns={'TEAM KICKOFF RETURNS:': 'Kickoff_returns'}, inplace=True)
dfs[37].rename(columns={'FG MISS RETURNS:': 'FG_miss_returns'}, inplace=True)
dfs[37].rename(columns={'KICK RETURN TDS': 'TDs'}, inplace=True)


#combine header and first row
dfs[37] = combine_header_and_first_row(dfs[37], 'Punt_returns')
dfs[37] = combine_header_and_first_row(dfs[37], 'Kickoff_returns')
dfs[37] = combine_header_and_first_row(dfs[37], 'FG_miss_returns')
dfs[37] = combine_header_and_first_row(dfs[37], 'TDs')

#set first row as header
dfs[37] = set_first_row_as_header(dfs[37])

#rename columns for clarity
dfs[37].rename(columns={'TD': 'Punt_returns_TD'}, inplace=True)
dfs[37].rename(columns={'30+': 'Punt_returns_30+'}, inplace=True)
dfs[37].rename(columns={'10+': 'Punt_returns_10+'}, inplace=True)
dfs[37].rename(columns={'40+': 'Kickoff_returns_40+'}, inplace=True)
dfs[37].rename(columns={'30': 'FG_miss_returns_30'}, inplace=True)


#split columns
dfs[37] = split_column(dfs[37], 'Punt_returns_No Punt_returns_Yds Punt_returns_Avg Punt_returns_LG')
dfs[37] = split_column(dfs[37], 'Kickoff_returns_No Kickoff_returns_Yds Kickoff_returns_Avg Kickoff_returns_LG Kickoff_returns_TD')
dfs[37] = split_column(dfs[37], 'FG_miss_returns_No FG_miss_returns_Yds FG_miss_returns_LG FG_miss_returns_TD')
dfs[37] = split_column(dfs[37], 'TDs_PR TDs_KOR TDs_FGM TDs_Tot')

#rename columns for clarity
dfs[37].rename(columns={'BP*': 'Long_gain_returns'}, inplace=True)
dfs[37].rename(columns={'TM': 'Team'}, inplace=True)

dfs[37]

,Team,Punt_returns_TD,Punt_returns_30+,Punt_returns_10+,Kickoff_returns_40+,Rk,30,Long_gain_returns,Punt_returns_No,Punt_returns_Yds,...,Kickoff_returns_LG,Kickoff_returns_TD,FG_miss_returns_No,FG_miss_returns_Yds,FG_miss_returns_LG,FG_miss_returns_TD,TDs_PR,TDs_KOR,TDs_FGM,TDs_Tot
0,BC,0,3,NaN,3,3,3,9,81,911,...,62,0,6,239,120,1,0,0,1,1
1,CGY,0,4,NaN,2,7,1,7,81,936,...,47,0,3,135,86,0,0,0,0,0
2,EDM,0,3,NaN,5,T4,1,9,78,802,...,98,1,4,80,30,0,0,1,0,1
3,HAM,0,4,NaN,8,T1,5,17,71,715,...,71,1,7,408,122,1,0,1,1,2
4,MTL,2,8,NaN,4,T1,2,14,81,937,...,79,0,5,221,125,1,2,0,1,3
5,OTT,1,8,NaN,2,6,1,11,87,991,...,93,1,4,98,69,0,1,1,0,2
6,SSK,3,4,NaN,3,T4,1,8,90,985,...,82,0,3,107,59,0,3,0,0,3
7,TOR,4,13,NaN,1,9,0,14,92,"1,330",...,41,0,4,34,23,0,4,0,0,4
8,WPG,1,3,NaN,3,8,2,8,83,921,...,43,0,6,227,102,0,1,0,0,1
9,CFL,11,50,340,31,NaN,16,97,744,"8,528",...,98,3,42,"1,549",125,3,11,3,3,17


In [2365]:
#clean opponent kick return df
#dfs[38]

index1 = 38
index2 = 42

# Ensure indexes are within the range of the list
if index1 < len(dfs) and index2 < len(dfs):
    # Swapping process
    temp = dfs[index1]  # Store the first item temporarily
    dfs[index1] = dfs[index2]  # Overwrite the first item with the second
    dfs[index2] = temp  # Assign the stored item to the second position
else:
    print("Error: Indexes out of range")


    
#drop unneeded columns
column_to_drop = 'CFL'
dfs[38] = dfs[38].drop(column_to_drop, axis=1)
column_to_drop = 'CFL.1'
dfs[38] = dfs[38].drop(column_to_drop, axis=1)

#rename columns for cleaning
dfs[38].rename(columns={'OPPONENT PUNT RETURNS:': 'Punt_returns'}, inplace=True)
dfs[38].rename(columns={'OPPONENT KICKOFF RETURNS:': 'Kickoff_returns'}, inplace=True)
dfs[38].rename(columns={'OPPT FG MISS RETS:': 'FG_miss_returns'}, inplace=True)
dfs[38].rename(columns={'OPPT KICK RET TDS': 'TDs'}, inplace=True)

#combine header and first row
dfs[38] = combine_header_and_first_row(dfs[38], 'Punt_returns')
dfs[38] = combine_header_and_first_row(dfs[38], 'Kickoff_returns')
dfs[38] = combine_header_and_first_row(dfs[38], 'FG_miss_returns')
dfs[38] = combine_header_and_first_row(dfs[38], 'TDs')

#set first row as header
dfs[38] = set_first_row_as_header(dfs[38])

#split columns
dfs[38] = split_column(dfs[38], 'Punt_returns_No Punt_returns_Yds Punt_returns_Avg Punt_returns_LG Punt_returns_TD Punt_returns_30+')
dfs[38] = split_column(dfs[38], 'Kickoff_returns_No Kickoff_returns_Yds Kickoff_returns_Avg Kickoff_returns_LG Kickoff_returns_TD Kickoff_returns_40+')
dfs[38] = split_column(dfs[38], 'FG_miss_returns_No FG_miss_returns_Yds FG_miss_returns_LG FG_miss_returns_TD FG_miss_returns_30')
dfs[38] = split_column(dfs[38], 'TDs_PR TDs_KOR TDs_FGM TDs_Tot')

#rename columns for clarity
dfs[38].rename(columns={'BP*': 'Long_gain_returns'}, inplace=True)
dfs[38].rename(columns={'TM': 'Team'}, inplace=True)

dfs[38]

,Team,10+,Long_gain_returns,Punt_returns_No,Punt_returns_Yds,Punt_returns_Avg,Punt_returns_LG,Punt_returns_TD,Punt_returns_30+,Kickoff_returns_No,...,Kickoff_returns_40+,FG_miss_returns_No,FG_miss_returns_Yds,FG_miss_returns_LG,FG_miss_returns_TD,FG_miss_returns_30,TDs_PR,TDs_KOR,TDs_FGM,TDs_Tot
0,BC,NaN,7,72,753,10.5,91,1,3,70,...,4,2,42,22,0,0,1,1,0,2
1,CGY,NaN,13,96,"1,099",11.4,94,4,8,57,...,2,7,246,102,0,3,4,0,0,4
2,EDM,NaN,12,85,"1,067",12.6,68,0,9,56,...,2,4,166,125,1,1,0,0,1,1
3,HAM,NaN,7,82,860,10.5,99,2,3,59,...,3,3,47,30,0,1,2,0,0,2
4,MTL,NaN,8,82,787,9.6,48,0,3,67,...,1,10,349,122,1,4,0,0,1,1
5,OTT,NaN,12,91,"1,015",11.2,107,2,8,57,...,1,4,282,120,1,3,2,0,1,3
6,SSK,NaN,11,89,"1,174",13.2,92,2,6,53,...,3,6,228,86,0,2,2,0,0,2
7,TOR,NaN,17,88,995,11.3,52,0,6,89,...,10,2,79,60,0,1,0,1,0,1
8,WPG,NaN,10,59,778,13.2,48,0,4,91,...,5,4,110,67,0,1,0,1,0,1
9,CFL,340,97,744,"8,528",11.5,107,11,50,599,...,31,42,"1,549",125,3,16,11,3,3,17


In [2366]:
#clean punting df
#dfs[41]



#rename columns for cleaning
dfs[41].rename(columns={'OPPONENT RETURNS': 'Oppt_returns'}, inplace=True)
dfs[41].rename(columns={'NET PUNTING*': 'Net'}, inplace=True)
dfs[41].rename(columns={'FIELD POSITION ADJUSTMENTS **': 'Adjusted'}, inplace=True)

#combine header and first row
dfs[41] = combine_header_and_first_row(dfs[41], 'Oppt_returns')
dfs[41] = combine_header_and_first_row(dfs[41], 'Net')
dfs[41] = combine_header_and_first_row(dfs[41], 'Adjusted')

#set first row as header
dfs[41] = set_first_row_as_header(dfs[41])

#rename columns for clarity
dfs[41].rename(columns={'TD': 'Oppt_returns_TD'}, inplace=True)
dfs[41].rename(columns={'30+': 'Oppt_returns_30+'}, inplace=True)
dfs[41].rename(columns={'Net_Avg Net_NetYd*': 'Net_Avg Net_Yd'}, inplace=True)
dfs[41].rename(columns={'Adjusted_Sgl* Adjusted_Cover Adjusted_Retn Adjusted_FP Adjusted_Yds Adjusted_Avg': 'Adjusted_Sgl Adjusted_Cover Adjusted_Retn Adjusted_FP_Yds Adjusted_Avg'}, inplace=True)

# #add zeroes for missing values
# dfs[41]['Adjusted_Sgl Adjusted_Cover Adjusted_Retn Adjusted_FP Adjusted_Yds Adjusted_Avg'] = dfs[41]['Adjusted_Sgl Adjusted_Cover Adjusted_Retn Adjusted_FP Adjusted_Yds Adjusted_Avg'].apply(lambda x: ' '.join(str(x).split() + ['0'] * (5 - len(str(x).split()))))


#split columns
dfs[41] = split_column(dfs[41], 'TM No Yards')
dfs[41] = split_column(dfs[41], 'Oppt_returns_No Oppt_returns_Yards Oppt_returns_Avg Oppt_returns_LG')
dfs[41] = split_column(dfs[41], 'Net_Avg Net_Yd')
dfs[41] = split_column(dfs[41], 'Adjusted_Sgl Adjusted_Cover Adjusted_Retn Adjusted_FP_Yds Adjusted_Avg')

#rename for clarity
dfs[41].rename(columns={'TM': 'Team'}, inplace=True)

#move team column to front
dfs[41] = dfs[41][['Team'] + [col for col in dfs[41].columns if col != 'Team']]

dfs[41]

,Team,Avg,LG,S,I10,Oppt_returns_TD,Oppt_returns_30+,Rnk,No,Yards,...,Oppt_returns_Yards,Oppt_returns_Avg,Oppt_returns_LG,Net_Avg,Net_Yd,Adjusted_Sgl,Adjusted_Cover,Adjusted_Retn,Adjusted_FP_Yds,Adjusted_Avg
0,BC,46.4,80,6,6,1,3,5,97,"4,497",...,753,10.5,91,36.1,"3,504",(61),-165,+101,"3,379",34.8
1,CGY,47.2,77,5,12,4,8,3,122,"5,761",...,"1,099",11.4,94,36.6,"4,462",(63),-190,+187,"4,396",36.0
2,EDM,45.4,85,9,5,0,9,9,121,"5,491",...,"1,067",12.6,68,33.6,"4,064",(108),-235,+140,"3,861",31.9
3,HAM,44.5,78,2,5,2,3,7,103,"4,586",...,860,10.5,99,35.4,"3,646",(18),-125,+90,"3,593",34.9
4,MTL,46.3,69,2,10,0,3,1,103,"4,773",...,787,9.6,48,37.9,"3,906",(9),-72,+49,"3,874",37.6
5,OTT,47.6,74,4,9,2,8,2,117,"5,568",...,"1,015",11.2,107,37.5,"4,393",(56),-180,+98,"4,255",36.4
6,SSK,47.9,90,6,9,2,6,6,117,"5,609",...,"1,174",13.2,92,35.9,"4,195",(45),-161,+55,"4,044",34.6
7,TOR,47.8,75,7,11,0,6,4,110,"5,260",...,995,11.3,52,36.2,"3,985",(101),-75,+68,"3,877",35.2
8,WPG,46.2,73,2,9,0,4,8,78,"3,602",...,778,13.2,48,35.2,"2,744",(2),-70,+70,"2,742",35.2
9,CFL,46.6,90,43,76,11,50,NaN,968,"45,147",...,"8,528",11.5,107,36.1,"34,899",(463),-1273,+858,"34,021",35.1


In [2367]:
#clean kickoff df
#dfs[42]



#rename columns for cleaning
dfs[42].rename(columns={'OPPONENT RETURNS': 'Oppt_returns'}, inplace=True)
dfs[42].rename(columns={'REGULAR K/Os: 2': 'Regular_K/Os'}, inplace=True)
dfs[42].rename(columns={"AVG OPP'T START YD LN 4": 'Oppt_start'}, inplace=True)


#combine header and first row
dfs[42] = combine_header_and_first_row(dfs[42], 'Oppt_returns')
dfs[42] = combine_header_and_first_row(dfs[42], 'Regular_K/Os')
dfs[42] = combine_header_and_first_row(dfs[42], 'Oppt_start')

#set first row as header
dfs[42] = set_first_row_as_header(dfs[42])

#rename columns for clarity
dfs[42].rename(columns={'TD': 'Oppt_returns_TD'}, inplace=True)
dfs[42].rename(columns={'40+': 'Oppt_returns_40+'}, inplace=True)
dfs[42].rename(columns={'Oppt_start_Poss Oppt_start_YdL Oppt_start_Av_YL Oppt_start_Rk': 'Oppt_start_Poss Oppt_start_YdL Oppt_start_Av_YL'}, inplace=True)
dfs[42].rename(columns={'Oppt_start_Poss Oppt_start_Yd Oppt_start_L4 Oppt_start_Av Oppt_start_YL Oppt_start_Rk': 'Oppt_start_Poss Oppt_start_YdL Oppt_start_Av_YL Oppt_start_Rk'}, inplace=True)
#dfs[42].rename(columns={'Oppt_start_Poss Oppt_start_YdL Oppt_start_Av_YL Oppt_start_Rk': 'Oppt_start_Poss Oppt_start_YdL Oppt_start_Av_YL'}, inplace=True)
dfs[42].rename(columns={'Oppt_start_Poss Oppt_start_YdL Oppt_start_Av_YL':'Oppt_start_Poss Oppt_start_YdL Oppt_start_Av_YL Oppt_start_Rk'}, inplace=True)

#split columns
dfs[42] = split_column(dfs[42],'TM No Yards Avg')
dfs[42] = split_column(dfs[42],'Oppt_returns_No Oppt_returns_Yards Oppt_returns_Avg Oppt_returns_LG')
dfs[42] = split_column(dfs[42],'Regular_K/Os_No Regular_K/Os_Yards Regular_K/Os_Avg')
dfs[42] = split_column(dfs[42],'Oppt_start_Poss Oppt_start_YdL Oppt_start_Av_YL Oppt_start_Rk')

#drop unneeded columns
column_to_drop = 'Oppt_start_Rk'
dfs[42] = dfs[42].drop(column_to_drop, axis=1)
column_to_drop = 'Oppt_start_Poss'
dfs[42] = dfs[42].drop(column_to_drop, axis=1)

#rename for clarity
dfs[42].rename(columns={'TM': 'Team'}, inplace=True)

#move team column to front
dfs[42] = dfs[42][['Team'] + [col for col in dfs[42].columns if col != 'Team']]

dfs[42]

/var/folders/pg/qyrtrfln0b9761tg47bv7rhc0000gn/T/ipykernel_88897/3824136787.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs[42].rename(columns={'TD': 'Oppt_returns_TD'}, inplace=True)
/var/folders/pg/qyrtrfln0b9761tg47bv7rhc0000gn/T/ipykernel_88897/3824136787.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs[42].rename(columns={'40+': 'Oppt_returns_40+'}, inplace=True)
/var/folders/pg/qyrtrfln0b9761tg47bv7rhc0000gn/T/ipykernel_88897/3824136787.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org

,Team,LG,S,Rec,Oppt_returns_TD,Oppt_returns_40+,Rank,No,Yards,Avg,Oppt_returns_No,Oppt_returns_Yards,Oppt_returns_Avg,Oppt_returns_LG,Regular_K/Os_No,Regular_K/Os_Yards,Regular_K/Os_Avg,Oppt_start_YdL,Oppt_start_Av_YL
0,BC,79,1,0,1,4,5,72,"4,384",60.9,70,"1,363",19.5,82,69,"4,338",62.9,"2,562",36.6
1,CGY,82,0,0,0,2,3,57,"3,653",64.1,57,"1,155",20.3,42,56,"3,638",65.0,"2,030",36.9
2,EDM,100,1,1,0,2,9,61,"3,534",57.9,56,918,16.4,62,61,"3,534",57.9,"2,297",38.9
3,HAM,79,1,0,0,3,7,64,"3,822",59.7,59,"1,221",20.7,49,61,"3,780",62.0,"2,506",40.4
4,MTL,80,0,0,0,1,4,68,"4,412",64.9,67,"1,452",21.7,42,68,"4,412",64.9,"2,402",36.4
5,OTT,77,0,0,0,1,8,59,"3,494",59.2,57,"1,106",19.4,50,58,"3,481",60.0,"2,354",39.9
6,SSK,95,1,3,0,3,6,58,"3,425",59.1,53,"1,017",19.2,43,54,"3,373",62.5,"2,050",37.3
7,TOR,100,2,0,1,10,1,91,"6,447",70.8,89,"2,314",26.0,93,91,"6,447",70.8,"3,128",35.1
8,WPG,90,1,0,1,5,2,92,"6,061",65.9,91,"2,256",24.8,98,92,"6,061",65.9,"3,367",37.4
9,CFL,100,7,4,3,31,NaN,622,"39,232",63.1,599,"12,802",21.4,98,610,"39,064",64.0,"22,696",37.5


In [2368]:
#clean special teams cover penalties df
#dfs[43]

#set first row as header
dfs[43] = set_first_row_as_header(dfs[43])

#drop unneeded columns
column_to_drop = 'Rk'
dfs[43] = dfs[43].drop(column_to_drop, axis=1)
column_to_drop = 'NOTES & LEGEND:'
dfs[43] = dfs[43].drop(column_to_drop, axis=1)
column_to_drop = 'Wk PR Yds Avg TD 30+'
dfs[43] = dfs[43].drop(column_to_drop, axis=1)
column_to_drop = '10+'
dfs[43] = dfs[43].drop(column_to_drop, axis=1)


#rename columns for cleaning/clarity
dfs[43].rename(columns={'PR KOR':'Penalties_punt Penalties_K/O'}, inplace=True)
dfs[43].rename(columns={'Tot PerG':'Total Per_game'}, inplace=True)


#split column
dfs[43] = split_column(dfs[43], 'Penalties_punt Penalties_K/O')
dfs[43] = split_column(dfs[43], 'Total Per_game')
#rename for clarity
dfs[43].rename(columns={'TM': 'Team'}, inplace=True)

dfs[43]

,Team,Penalties_punt,Penalties_K/O,Total,Per_game
0,BC,21,3,24,1.33
1,CGY,13,6,19,1.06
2,EDM,11,7,18,1.00
3,HAM,20,4,24,1.33
4,MTL,22,8,30,1.67
5,OTT,11,4,15,0.83
6,SSK,7,6,13,0.72
7,TOR,11,2,13,0.72
8,WPG,13,0,13,0.72
9,CFL,129,40,169,2.09


In [2369]:
import os

index_to_filename = {
    0: "game_stat_trends",
    2: "scoring_breakdown",
    4: "team_scoring",
    5: "opponent_scoring",
    6: "turnover_analysis",
    8: "possesion_analysis",
    10: "opponent_possession_analysis",
    11: "time_of_possesion_field_position",
    13: "big_play_analysis",
    14: "red_zone_results",
    17: "net_offence",
    18: "opponent_net_offence",
    19: "first_down_offence",
    20: "first_downs_made",
    21: "rushing_analysis",
    22: "passing_analysis_base_data",
    24: "passing_analysis_range_data",
    25: "opponent_passing_analysis_base_data",
    26: "opponent_passing_analysis_range_data",
    27: "second_down_conversions",
    28: "opponent_second_down_conversions",
    30: "third_and_short",
    33: "penalties",
    34: "field_goals",
    36: "converts",
    37: "kick_returns",
    38: "opponent_kick_return",
    41: "punting_analysis",
    42: "kickoff_analysis",
    43: "special_teams_cover_penalites"
}

folder_name = "week_21_2023"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)


# Loop through the dictionary and save each DataFrame
for index, filename in index_to_filename.items():
    df = dfs[index]
    file_path = os.path.join(folder_name, f"{filename}.csv")
    df.to_csv(file_path, index=False)

print("DataFrames saved in the 'week_21_2023' folder!")

DataFrames saved in the 'week_21_2023' folder!


In [230]:
base_dir = "/Users/jaredboretsky/Documents/concordia-bootcamps/ds-final_project/CFL_Data/week_{}_2023/"
file_name = "third_and_short.csv"



# Loop through weeks 1 to 21
for week in range(12, 22):
    # Construct the full file path
    file_path = base_dir.format(week) + file_name

    try:
        # Load the DataFrame from CSV
        df = pd.read_csv(file_path)
        df.rename(columns={'TM':'Team'}, inplace=True)
    
        # Save the updated DataFrame back to CSV
        df.to_csv(file_path, index=False)

        print(f"Week {week}: File processed and saved successfully.")

    except Exception as e:
        print(f"Week {week}: An error occurred - {e}")

Week 12: File processed and saved successfully.
Week 13: File processed and saved successfully.
Week 14: File processed and saved successfully.
Week 15: File processed and saved successfully.
Week 16: File processed and saved successfully.
Week 17: File processed and saved successfully.
Week 18: File processed and saved successfully.
Week 19: File processed and saved successfully.
Week 20: File processed and saved successfully.
Week 21: File processed and saved successfully.


In [224]:
file_path = '/Users/jaredboretsky/Documents/concordia-bootcamps/ds-final_project/CFL_Data/week_3_2023/game_stat_trends.csv'
df = pd.read_csv(file_path)

In [225]:
df

,TEAM,GP,Record,Record.1,Home,Home.1,Away,Away.1,Scoring,Scoring.1,Scoring.2
0,BC,3,3,0,1,0,2,0,77.0,21.0,56.0
1,Calgary,3,1,2,0,2,1,0,67.0,69.0,-2.0
2,Edmonton,3,0,3,0,2,0,1,44.0,82.0,-38.0
3,Hamilton,3,0,3,0,1,0,2,57.0,112.0,-55.0
4,Montréal,2,2,0,1,0,1,0,57.0,24.0,33.0
5,Ottawa,2,0,2,0,1,0,1,27.0,45.0,-18.0
6,Saskatchewan,3,2,1,0,1,2,0,73.0,84.0,-11.0
7,Toronto,2,2,0,1,0,1,0,75.0,45.0,30.0
8,Winnipeg,3,2,1,1,1,1,0,93.0,88.0,5.0
9,2023 Total,12,12,12,4,8,8,4,47.5,NaN,NaN


In [232]:
#drop unwanted columns
file_path = '/Users/jaredboretsky/Documents/concordia-bootcamps/ds-final_project/CFL_Data/week_2_2023/second_down_conversions.csv'

df = pd.read_csv(file_path)

In [253]:





#drop unneeded columns





#rename columns for clarity
df.rename(columns={'TM': 'Team'}, inplace=True)

df

,1-3_yds_%,4-6_yds_%,7+_yds_%,Team,Att,Md,%,1-3_yds_Att,1-3_yds_Md,4-6_yds_Att,4-6_yds_Md,7+_yds_Att,7+_yds_Md,Yds_to_go_Yards,Yds_to_go_Avg
0,81.8,61.9,29.4,BC,49,27,55.1,11,9,21,13,17,5,296,6.04
1,80.0,33.3,36.0,CGY,47,21,44.7,10,8,12,4,25,9,337,7.17
2,50.0,60.0,13.6,EDM,35,10,28.6,8,4,5,3,22,3,287,8.20
3,80.0,60.0,20.0,HAM,40,15,37.5,5,4,10,6,25,5,297,7.43
4,66.7,40.0,35.7,MTL,22,9,40.9,3,2,5,2,14,5,183,8.32
5,71.4,37.5,22.2,OTT,42,14,33.3,7,5,8,3,27,6,307,7.31
6,50.0,62.5,50.0,SSK,48,25,52.1,10,5,8,5,30,15,356,7.42
7,66.7,50.0,41.7,TOR,19,9,47.4,3,2,4,2,12,5,170,8.95
8,75.0,68.8,54.5,WPG,46,29,63.0,8,6,16,11,22,12,305,6.63
9,69.2,55.1,33.5,CFL,348,159,45.7,65,45,89,49,194,65,"2,538",7.29


In [254]:
df.to_csv(file_path, index=False)